In [ ]:
import requests
import json
import re

# URLs for live data
ipowatch_url = 'https://ipowatch.in/ipo-grey-market-premium-latest-ipo-gmp/'
investorgain_url = 'https://www.investorgain.com/report/live-ipo-gmp/331/'

# --- STEP 1: Fetch HTML content (truncate to stay within LLM context) ---
def fetch_html(url, limit=20000):
    try:
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()
        text = re.sub(r'\s+', ' ', resp.text)  # collapse extra whitespace
        return text[:limit]
    except Exception as e:
        print(f"⚠️ Failed to fetch {url}: {e}")
        return ""

ipowatch_html = fetch_html(ipowatch_url)
investorgain_html = fetch_html(investorgain_url)

# --- STEP 2: Build the LLM prompt ---
prompt = f"""
You are a financial data parser.

You will be given two HTML snippets from different IPO tracking websites.
Extract and compare the IPO listings into a **Markdown table** with these exact columns:

| IPO | Category | GMP_InvestorGain | GMP_IPOWatch | Issue_Price | Open_Date | Close_Date | GMP_Diff |

- GMP_Diff = GMP_InvestorGain - GMP_IPOWatch
- Treat missing GMP as 0.
- The table must start with a header row and pipes (|).
- Do not include any explanations, disclaimers, or text outside the table.

### HTML from IPOWatch:
{ipowatch_html}

### HTML from InvestorGain:
{investorgain_html}
"""

# --- STEP 3: Send to Perplexity API ---
api_url = "https://api.perplexity.ai/chat/completions"
headers = {
    "Authorization": "Bearer your pplx key",
    "Content-Type": "application/json"
}
payload = {
    "model": "sonar-pro",          # model used
    "temperature": 0.1,            # low temp for consistency
    "max_tokens": 2500,            # more room for large tables
    "messages": [
        {"role": "system", "content": "You are a structured data extraction assistant."},
        {"role": "user", "content": prompt}
    ]
}

# --- STEP 4: Send request ---
try:
    response = requests.post(api_url, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    result = response.json()

    # Extract model output safely
    content = result.get("choices", [{}])[0].get("message", {}).get("content", "")

    # --- STEP 5: Sanitize output (only keep markdown table) ---
    match = re.search(r"(\|.+\|[\s\S]*)", content)
    if match:
        table = match.group(1).strip()
        print(table)
        # Optionally save
        with open("ipo_comparison_table.md", "w", encoding="utf-8") as f:
            f.write(table)
        print("\n✅ Markdown table saved to ipo_comparison_table.md")
    else:
        print("⚠️ No valid table found in response.\nRaw content:")
        print(content)

except requests.exceptions.RequestException as e:
    print(f"HTTP Error: {e}")
except (KeyError, json.JSONDecodeError) as e:
    print("⚠️ Parsing Error:", e)
    print("Raw response:", response.text)


| IPO                        | Category   | GMP_InvestorGain | GMP_IPOWatch | Issue_Price | Open_Date   | Close_Date  | GMP_Diff |
|----------------------------|------------|------------------|--------------|-------------|-------------|-------------|----------|
| Tata Technologies          | Mainboard  | 400              | 412          | 500         | 2023-11-22  | 2023-11-24  | -12      |
| Gandhar Oil Refinery       | Mainboard  | 78               | 82           | 169         | 2023-11-22  | 2023-11-24  | -4       |
| Flair Writing Industries   | Mainboard  | 75               | 76           | 304         | 2023-11-22  | 2023-11-24  | -1       |
| Fedbank Financial Services | Mainboard  | 6                | 7            | 140         | 2023-11-22  | 2023-11-24  | -1       |
| Indian Renewable Energy    | SME        | 10               | 12           | 60          | 2023-11-21  | 2023-11-23  | -2       |
| Accent Microcell           | SME        | 110              | 120          | 140  

In [ ]:
import requests
import json
import re

# URLs
ipowatch_url = 'https://ipowatch.in/ipo-grey-market-premium-latest-ipo-gmp/'
investorgain_url = 'https://www.investorgain.com/report/live-ipo-gmp/331/'

# --- Fetch HTML ---
def fetch_html(url, limit=15000):  # smaller limit -> top section only (latest IPOs)
    try:
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()
        html = re.sub(r'\s+', ' ', resp.text)
        return html[:limit]
    except Exception as e:
        print(f"⚠️ Failed to fetch {url}: {e}")
        return ""

ipowatch_html = fetch_html(ipowatch_url)
investorgain_html = fetch_html(investorgain_url)

# --- Enhanced Prompt ---
prompt = f"""
You are a financial data extractor.

Extract **only the current and upcoming IPOs** (ignore old or archived ones) from the two HTML pages below.
Focus on IPOs mentioned near the top of the page (recent issues).

Output a **Markdown table** with the exact columns:

| IPO | Category | GMP_InvestorGain | GMP_IPOWatch | Issue_Price | Open_Date | Close_Date | GMP_Diff |

Rules:
- GMP_Diff = GMP_InvestorGain - GMP_IPOWatch
- If GMP is missing, assume 0.
- Only include IPOs that are currently open, recently closed, or upcoming.
- Ignore historical performance tables or 2023 data.
- Sort IPOs by **Open_Date descending (newest first)**.
- Output table only. No explanations, notes, or extra text.

### IPOWatch HTML (latest section only):
{ipowatch_html}

### InvestorGain HTML (latest section only):
{investorgain_html}
"""

# --- Send to Perplexity API ---
api_url = "https://api.perplexity.ai/chat/completions"
headers = {
    "Authorization": "Bearer your pplx key",
    "Content-Type": "application/json"
}
payload = {
    "model": "sonar-pro",
    "temperature": 0.1,
    "max_tokens": 3000,
    "messages": [
        {"role": "system", "content": "You are a precise data extraction agent."},
        {"role": "user", "content": prompt}
    ]
}

# --- Handle Request ---
try:
    response = requests.post(api_url, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    result = response.json()
    content = result.get("choices", [{}])[0].get("message", {}).get("content", "")

    # Extract table safely
    match = re.search(r"(\|.+\|[\s\S]*)", content)
    if match:
        table = match.group(1).strip()
        with open("latest_ipo_comparison.md", "w", encoding="utf-8") as f:
            f.write(table)
        print(table)
        print("\n✅ Saved latest IPOs to latest_ipo_comparison.md")
    else:
        print("⚠️ No table found. Raw output:")
        print(content)

except Exception as e:
    print(f"❌ Error: {e}")


| IPO                       | Category   | GMP_InvestorGain | GMP_IPOWatch | Issue_Price      | Open_Date   | Close_Date  | GMP_Diff |
|---------------------------|------------|------------------|--------------|------------------|-------------|-------------|----------|
| Mahamaya Lifesciences     | SME        | 18               | 15           | ₹108 – ₹114      | 2025-11-11  | 2025-11-13  | 3        |
| Workmates Core2Cloud      | SME        | 22               | 20           | ₹200 – ₹204      | 2025-11-11  | 2025-11-13  | 2        |
| PhysicsWallah             | Mainboard  | 35               | 30           | ₹103 – ₹109      | 2025-11-11  | 2025-11-13  | 5        |
| Emmvee Photovoltaic Power | Mainboard  | 40               | 38           | ₹206 – ₹217      | 2025-11-11  | 2025-11-13  | 2        |
| Tenneco Clean Air         | Mainboard  | 45               | 42           | ₹378 – ₹397      | 2025-11-12  | 2025-11-14  | 3        |
| Shining Tools             | SME        | 12          

In [ ]:
import re
import os
import json
from datetime import datetime
from dateutil import parser as dateparse
from io import StringIO

import pandas as pd
from pymongo import MongoClient
from transformers import pipeline

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_SENT = "ipo_sentiment"

# Sentiment model: CPU-friendly
SENTIMENT_MODEL = "distilbert-base-uncased-finetuned-sst-2-english"
# --------------------------------

# ---------- helper parsers ----------
def markdown_table_to_df(table_md: str) -> pd.DataFrame:
    """
    Converts a GitHub-style markdown table string into a pandas DataFrame.
    """
    # Remove leading/trailing whitespace
    table_md = table_md.strip()
    # Find header divider (---) and split
    lines = [ln.strip() for ln in table_md.splitlines() if ln.strip()]
    # Keep only lines starting with '|'
    lines = [ln for ln in lines if ln.startswith("|")]
    if len(lines) < 2:
        raise ValueError("Not a valid markdown table.")
    header = [h.strip() for h in lines[0].strip("|").split("|")]
    rows = []
    for ln in lines[2:]:  # skip header and separator
        cols = [c.strip() for c in ln.strip("|").split("|")]
        # pad if missing
        if len(cols) < len(header):
            cols += [""] * (len(header) - len(cols))
        rows.append(cols)
    df = pd.DataFrame(rows, columns=header)
    return df

def clean_money(x):
    if x is None:
        return None
    s = str(x).strip()
    s = s.replace("₹", "").replace(",", "").replace(" ", "")
    if s in ["", "-", "—", "nan", "None"]:
        return None
    # handle ranges like 206-217 or 206-217 or 206-217
    if "-" in s:
        parts = [p for p in s.split("-") if p != ""]
        try:
            nums = [float(re.sub(r"[^\d.]", "", p)) for p in parts]
            return {"min": min(nums), "max": max(nums), "mid": sum(nums)/len(nums)}
        except:
            return s
    # single number
    try:
        return {"min": float(s), "max": float(s), "mid": float(s)}
    except:
        # maybe has percent or extra chars
        m = re.search(r"(-?\d+(\.\d+)?)", s)
        if m:
            return {"min": float(m.group(1)), "max": float(m.group(1)), "mid": float(m.group(1))}
    return None

def parse_gmp(x):
    # allow strings like '65', '0', '₹65', '65 (16.62%)'
    if x is None:
        return None
    s = str(x).strip()
    if s == "" or s.lower() in ["nan", "-"]:
        return None
    # find first number (allow negative)
    m = re.search(r"(-?\d+(\.\d+)?)", s.replace(",", ""))
    if m:
        return float(m.group(1))
    return None

def parse_date(x):
    if x is None or x == "":
        return None
    try:
        # some dates are like 2025-11-12; others can be textual
        d = dateparse.parse(x, dayfirst=False, fuzzy=True)
        return d.date().isoformat()
    except Exception:
        return None

# ---------- Mongo utilities ----------
def get_mongo_col(uri=MONGO_URI, dbname=DB_NAME, colname=COL_IPOS):
    client = MongoClient(uri, serverSelectionTimeoutMS=5000)
    db = client[dbname]
    col = db[colname]
    return col

# ---------- Step A: Read markdown table (from file or string) ----------
def load_table_from_file(path="latest_ipo_comparison.md"):
    if not os.path.exists(path):
        raise FileNotFoundError(f"{path} not found")
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

# ---------- Step B: Convert -> normalized DataFrame ----------
def normalize_ipos(table_md: str) -> pd.DataFrame:
    df = markdown_table_to_df(table_md)

    # Standardize column names (strip spaces)
    df.columns = [c.strip() for c in df.columns]

    # Ensure all expected columns exist
    expected = ["IPO","Category","GMP_InvestorGain","GMP_IPOWatch","Issue_Price","Open_Date","Close_Date","GMP_Diff"]
    for e in expected:
        if e not in df.columns:
            df[e] = None

    # Parse numeric GMPs
    df["GMP_InvestorGain_num"] = df["GMP_InvestorGain"].apply(parse_gmp)
    df["GMP_IPOWatch_num"] = df["GMP_IPOWatch"].apply(parse_gmp)

    # If GMP_Diff absent or zero, recompute
    def compute_diff(row):
        a = row["GMP_InvestorGain_num"]
        b = row["GMP_IPOWatch_num"]
        if pd.isna(a) and pd.isna(b):
            return None
        a = 0.0 if a is None else float(a)
        b = 0.0 if b is None else float(b)
        return a - b

    df["GMP_Diff_num"] = df.apply(lambda r: compute_diff(r) if (str(r.get("GMP_Diff")).strip() in ["", "None", "0"]) else float(re.search(r"(-?\d+(\.\d+)?)", str(r["GMP_Diff"])).group(1)), axis=1)

    # Normalize Issue_Price into structured fields
    df["Issue_Price_struct"] = df["Issue_Price"].apply(clean_money)

    # Normalize dates
    df["Open_Date_iso"] = df["Open_Date"].apply(parse_date)
    df["Close_Date_iso"] = df["Close_Date"].apply(parse_date)

    # Build final payload per IPO (dict)
    records = []
    for _, r in df.iterrows():
        rec = {
            "ipo": r["IPO"],
            "category": r["Category"],
            "gmp_investorgain": None if pd.isna(r["GMP_InvestorGain"]) else r["GMP_InvestorGain"],
            "gmp_investorgain_num": r["GMP_InvestorGain_num"],
            "gmp_ipowatch": None if pd.isna(r["GMP_IPOWatch"]) else r["GMP_IPOWatch"],
            "gmp_ipowatch_num": r["GMP_IPOWatch_num"],
            "gmp_diff": r["GMP_Diff_num"],
            "issue_price_raw": r["Issue_Price"],
            "issue_price": r["Issue_Price_struct"],
            "open_date": r["Open_Date_iso"],
            "close_date": r["Close_Date_iso"],
            "source_created_at": datetime.utcnow().isoformat()
        }
        records.append(rec)
    return pd.DataFrame(records)

# ---------- Step C: Insert into MongoDB (upsert by IPO + open_date) ----------
def upsert_ipos_to_mongo(df_norm: pd.DataFrame, uri=MONGO_URI):
    col = get_mongo_col(uri, DB_NAME, COL_IPOS)
    inserted = 0
    for _, row in df_norm.iterrows():
        key = {"ipo": row["ipo"], "open_date": row["open_date"]}
        doc = dict(row.dropna().to_dict())
        # Upsert: update existing doc or insert new
        col.update_one(key, {"$set": doc, "$setOnInsert": {"created_at": datetime.utcnow()}}, upsert=True)
        inserted += 1
    print(f"Upserted {inserted} IPO records to MongoDB collection '{COL_IPOS}'.")

# ---------- Step D: Sentiment pipeline (bootstrap) ----------
def init_sentiment_pipeline(model_name=SENTIMENT_MODEL):
    # device=-1 ensures CPU usage
    nlp = pipeline("sentiment-analysis", model=model_name, device=-1)
    return nlp

def analyze_and_store_sentiment(ipo_name: str, texts: list, nlp, uri=MONGO_URI):
    """
    texts: list of textual snippets (news headlines, tweets, etc.) about the IPO
    performs sentiment analysis (per snippet), aggregates, and stores result in mongo.
    """
    col_sent = get_mongo_col(uri, DB_NAME, COL_SENT)
    # run model in batches
    outs = nlp(texts, truncation=True)
    # outs is list of dicts with 'label' (POSITIVE/NEGATIVE) and 'score'
    # Convert to numeric sentiment: positive -> +score, negative -> -score
    scores = []
    for o in outs:
        label = o["label"].upper()
        s = float(o["score"])
        val = s if label == "POSITIVE" else -s
        scores.append(val)
    # Aggregations
    avg_sent = sum(scores) / len(scores) if scores else 0.0
    weighted_sent = avg_sent  # placeholder - you can add weights by source, recency, etc.
    doc = {
        "ipo": ipo_name,
        "samples_count": len(texts),
        "avg_sentiment": avg_sent,
        "weighted_sentiment": weighted_sent,
        "per_sample": list(zip(texts, scores)),
        "analyzed_at": datetime.utcnow().isoformat()
    }
    # Upsert by ipo
    col_sent.update_one({"ipo": ipo_name}, {"$set": doc}, upsert=True)
    return doc

# ---------- MAIN ----------
if __name__ == "__main__":
    # 1) Load markdown table output produced earlier
    mdpath = "latest_ipo_comparison.md"
    md = load_table_from_file(mdpath)
    # 2) Normalize into structured DataFrame
    df_norm = normalize_ipos(md)
    print("Normalized rows:", len(df_norm))
    # 3) Upsert IPOs to MongoDB
    upsert_ipos_to_mongo(df_norm)
    # 4) Initialize sentiment model (CPU)
    nlp = init_sentiment_pipeline()
    print("Sentiment model loaded (CPU).")
    # 5) Example: analyze sample texts per IPO (replace with real scrapes later)
    # Here we just show a scaffold: you will replace the texts list with real headlines/tweets/news.
    example_texts = [
        "Investors bullish on Tenneco Clean Air IPO after strong subscription.",
        "Some analysts warn valuation high for Tenneco listing."
    ]
    # Analyze only first IPO for demo
    if not df_norm.empty:
        first_ipo = df_norm.iloc[0]["ipo"]
        res = analyze_and_store_sentiment(first_ipo, example_texts, nlp)
        print("Sentiment stored for", first_ipo, ":", res["avg_sentiment"])
    else:
        print("No IPO rows to analyze.")


In [6]:
import re
import os
from datetime import datetime
from dateutil import parser as dateparse
import pandas as pd
from pymongo import MongoClient

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
# -----------------------------

# ---------- Utility Functions ----------
def markdown_table_to_df(table_md: str) -> pd.DataFrame:
    """Convert a GitHub-style Markdown table into a pandas DataFrame."""
    lines = [ln.strip() for ln in table_md.splitlines() if ln.strip()]
    lines = [ln for ln in lines if ln.startswith("|")]
    if len(lines) < 2:
        raise ValueError("Invalid markdown table format.")
    header = [h.strip() for h in lines[0].strip("|").split("|")]
    rows = []
    for ln in lines[2:]:
        cols = [c.strip() for c in ln.strip("|").split("|")]
        if len(cols) < len(header):
            cols += [""] * (len(header) - len(cols))
        rows.append(cols)
    df = pd.DataFrame(rows, columns=header)
    return df


def parse_float(value):
    """Extract first numeric float value from a string like ₹123 or '5.5%'."""
    if not value or str(value).strip() in ["-", "—", "None"]:
        return None
    try:
        match = re.search(r"(-?\d+(\.\d+)?)", str(value).replace(",", ""))
        return float(match.group(1)) if match else None
    except Exception:
        return None


def clean_issue_price(price_str):
    """Extract range or single numeric price."""
    if not price_str:
        return None
    s = str(price_str).replace("₹", "").replace(",", "").strip()
    if "-" in s:
        parts = re.findall(r"\d+\.?\d*", s)
        if len(parts) >= 2:
            low, high = float(parts[0]), float(parts[1])
            return {"min": low, "max": high, "avg": (low + high) / 2}
    num = parse_float(s)
    if num is not None:
        return {"min": num, "max": num, "avg": num}
    return None


def parse_date(date_str):
    """Normalize dates to ISO format (YYYY-MM-DD)."""
    if not date_str or date_str.strip() == "":
        return None
    try:
        d = dateparse.parse(date_str, fuzzy=True)
        return d.date().isoformat()
    except Exception:
        return None


def get_mongo_collection(uri, db_name, collection):
    client = MongoClient(uri, serverSelectionTimeoutMS=5000)
    db = client[db_name]
    return db[collection]


# ---------- Core Processing ----------
def normalize_ipo_table(table_md: str) -> pd.DataFrame:
    """Parse markdown table, clean and normalize IPO data."""
    df = markdown_table_to_df(table_md)
    df.columns = [c.strip() for c in df.columns]

    # Ensure all expected columns exist
    expected_cols = [
        "IPO", "Category", "GMP_InvestorGain", "GMP_IPOWatch",
        "Issue_Price", "Open_Date", "Close_Date", "GMP_Diff"
    ]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = None

    # Clean numeric and date columns
    df["GMP_InvestorGain_num"] = df["GMP_InvestorGain"].apply(parse_float)
    df["GMP_IPOWatch_num"] = df["GMP_IPOWatch"].apply(parse_float)

    # If GMP_Diff missing or invalid, recompute it
    def compute_diff(row):
        a, b = row["GMP_InvestorGain_num"], row["GMP_IPOWatch_num"]
        return (a or 0) - (b or 0)

    df["GMP_Diff_num"] = [
        parse_float(x) if parse_float(x) is not None else compute_diff(r)
        for x, r in zip(df["GMP_Diff"], df.to_dict(orient="records"))
    ]

    df["Issue_Price_struct"] = df["Issue_Price"].apply(clean_issue_price)
    df["Open_Date_iso"] = df["Open_Date"].apply(parse_date)
    df["Close_Date_iso"] = df["Close_Date"].apply(parse_date)

    # Construct normalized structure
    records = []
    for _, row in df.iterrows():
        rec = {
            "ipo": row["IPO"],
            "category": row["Category"],
            "gmp_investorgain": row["GMP_InvestorGain_num"],
            "gmp_ipowatch": row["GMP_IPOWatch_num"],
            "gmp_diff": row["GMP_Diff_num"],
            "issue_price": row["Issue_Price_struct"],
            "open_date": row["Open_Date_iso"],
            "close_date": row["Close_Date_iso"],
            "raw": {
                "GMP_InvestorGain": row["GMP_InvestorGain"],
                "GMP_IPOWatch": row["GMP_IPOWatch"],
                "Issue_Price": row["Issue_Price"],
                "GMP_Diff": row["GMP_Diff"],
            },
            "inserted_at": datetime.utcnow().isoformat()
        }
        records.append(rec)
    return pd.DataFrame(records)


def insert_to_mongodb(df: pd.DataFrame):
    """Insert or update IPO records in MongoDB."""
    collection = get_mongo_collection(MONGO_URI, DB_NAME, COL_IPOS)
    inserted, updated = 0, 0
    for _, row in df.iterrows():
        key = {"ipo": row["ipo"], "open_date": row["open_date"]}
        existing = collection.find_one(key)
        if existing:
            collection.update_one(key, {"$set": row.to_dict()})
            updated += 1
        else:
            collection.insert_one(row.to_dict())
            inserted += 1
    print(f"✅ Inserted {inserted}, Updated {updated} IPO records in MongoDB.")


# ---------- Main Entry ----------
if __name__ == "__main__":
    md_path = "latest_ipo_comparison.md"
    if not os.path.exists(md_path):
        raise FileNotFoundError("⚠️ File latest_ipo_comparison.md not found. Run your LLM extractor first.")
    with open(md_path, "r", encoding="utf-8") as f:
        md_content = f.read()

    df_norm = normalize_ipo_table(md_content)
    print("🧾 Normalized IPO entries:", len(df_norm))
    insert_to_mongodb(df_norm)


🧾 Normalized IPO entries: 9
✅ Inserted 0, Updated 9 IPO records in MongoDB.


In [ ]:
import os
import re
import json
import requests
import pandas as pd
from pymongo import MongoClient
from time import sleep

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
OUTPUT_DIR = "ipo_details"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Perplexity API Config
PPLX_API_URL = "https://api.perplexity.ai/chat/completions"
PPLX_API_KEY = os.getenv("PPLX_API_KEY", "your pplx key")  # <-- put your key here
MODEL_NAME = "sonar-pro"
# -------------------------------------------


def get_mongo_collection(uri, db_name, collection):
    client = MongoClient(uri, serverSelectionTimeoutMS=5000)
    db = client[db_name]
    return db[collection]


def fetch_html(url, limit=80000):
    """Fetch the IPO HTML page with truncation."""
    try:
        resp = requests.get(url, timeout=30, headers={"User-Agent": "Mozilla/5.0"})
        resp.raise_for_status()
        html = re.sub(r"\s+", " ", resp.text)
        return html[:limit]
    except Exception as e:
        print(f"⚠️ Failed to fetch {url}: {e}")
        return ""


def generate_prompt(ipo_name, html):
    """Construct prompt for Perplexity extraction."""
    prompt = f"""
You are a financial data extraction model.

Extract the following 13 fields from the IPO details page HTML provided below.

Output as a **Markdown table** with these exact columns (one row only):

| IPO | Price Band | Issue Size | Issue Type | Listing Exchanges | IPO Dates | Market Lot & Amounts | Investor Quota Split | Anchor Details | Promoter Holdings (Pre/Post) | Financial Performance (FY23–FY25) | Valuation Ratios (EPS, ROE, ROCE, D/E, NAV) | Lead Managers & Registrar | Company Overview | Peer Comparison |

Rules:
- Extract exact numeric and date values from the HTML.
- Keep it concise and clean (no commentary).
- If a value is missing, leave the cell blank.
- All data must come from the provided HTML only.

### HTML for {ipo_name}:
{html}
"""
    return prompt


def call_perplexity(prompt):
    """Send HTML prompt to Perplexity API."""
    headers = {
        "Authorization": f"Bearer {PPLX_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL_NAME,
        "temperature": 0.1,
        "max_tokens": 4000,
        "messages": [
            {"role": "system", "content": "You are a precise structured data extractor for IPO information."},
            {"role": "user", "content": prompt}
        ]
    }

    try:
        resp = requests.post(PPLX_API_URL, headers=headers, json=payload, timeout=90)
        resp.raise_for_status()
        data = resp.json()
        content = data.get("choices", [{}])[0].get("message", {}).get("content", "")
        return content.strip()
    except Exception as e:
        print(f"❌ API error: {e}")
        return ""


def extract_table(markdown_text):
    """Extract table markdown only."""
    match = re.search(r"(\|.+\|[\s\S]*)", markdown_text)
    if match:
        return match.group(1).strip()
    return ""


def process_all_ipos():
    """Iterate through MongoDB IPOs, fetch IPOWatch page, call Perplexity."""
    collection = get_mongo_collection(MONGO_URI, DB_NAME, COL_IPOS)
    ipos = list(collection.find({}))
    print(f"📊 Found {len(ipos)} IPO records in MongoDB")

    for ipo in ipos:
        name = ipo.get("ipo", "").strip()
        if not name:
            continue

        # Create IPOWatch URL (format-safe)
        slug = name.lower().replace(" ", "-")
        url = f"https://ipowatch.in/{slug}-ipo-date-review-price-allotment-details/"
        print(f"\n🔍 Processing: {name}")
        print(f"🌐 {url}")

        html = fetch_html(url)
        if not html:
            continue

        prompt = generate_prompt(name, html)
        result = call_perplexity(prompt)
        table = extract_table(result)

        if table:
            out_path = os.path.join(OUTPUT_DIR, f"{slug}_details.md")
            with open(out_path, "w", encoding="utf-8") as f:
                f.write(table)
            print(f"✅ Saved extracted data for {name} → {out_path}")
        else:
            print(f"⚠️ No table extracted for {name}")

        # Sleep between API calls (to respect rate limits)
        sleep(3)


if __name__ == "__main__":
    process_all_ipos()


📊 Found 10 IPO records in MongoDB

🔍 Processing: PhysicsWallah
🌐 https://ipowatch.in/physicswallah-ipo-date-review-price-allotment-details/
✅ Saved extracted data for PhysicsWallah → ipo_details\physicswallah_details.md

🔍 Processing: Emmvee Photovoltaic Power
🌐 https://ipowatch.in/emmvee-photovoltaic-power-ipo-date-review-price-allotment-details/
⚠️ Failed to fetch https://ipowatch.in/emmvee-photovoltaic-power-ipo-date-review-price-allotment-details/: 404 Client Error: Not Found for url: https://ipowatch.in/emmvee-photovoltaic-power-ipo-date-review-price-allotment-details/

🔍 Processing: Tenneco Clean Air India
🌐 https://ipowatch.in/tenneco-clean-air-india-ipo-date-review-price-allotment-details/
⚠️ Failed to fetch https://ipowatch.in/tenneco-clean-air-india-ipo-date-review-price-allotment-details/: 404 Client Error: Not Found for url: https://ipowatch.in/tenneco-clean-air-india-ipo-date-review-price-allotment-details/

🔍 Processing: Mahamaya Lifesciences
🌐 https://ipowatch.in/mahamay

In [ ]:
import os
import re
import json
import requests
from datetime import datetime
from pymongo import MongoClient
from concurrent.futures import ThreadPoolExecutor, as_completed

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"

PPLX_API_KEY = os.getenv("PPLX_API_KEY", "your pplx key")  # 👈 put your key here
PPLX_API_URL = "https://api.perplexity.ai/chat/completions"
MODEL_NAME = "sonar-pro"

MAX_WORKERS = 2   # number of IPOs to run in parallel
# -------------------------------------------


# ---------- UTILITIES ----------
def make_two_word_slug(name: str) -> str:
    """Create IPOWatch slug using first two words of IPO name."""
    tokens = re.findall(r"[A-Za-z0-9]+", name)
    return "-".join(t.lower() for t in tokens[:2]) if tokens else ""


def fetch_html(url: str, limit: int = 70000) -> str:
    """Fetch IPOWatch HTML page."""
    try:
        resp = requests.get(url, timeout=25, headers={"User-Agent": "Mozilla/5.0"})
        resp.raise_for_status()
        html = re.sub(r"\s+", " ", resp.text)
        return html[:limit]
    except Exception as e:
        return f"⚠️ Fetch failed: {e}"


def generate_prompt(ipo_name: str, html: str) -> str:
    """Generate extraction prompt for Perplexity."""
    return f"""
You are a financial data extraction model.

Extract the following 13 fields from the IPO details page HTML provided below.

Output a Markdown table with these exact columns (one row only):

| IPO | Price Band | Issue Size | Issue Type | Listing Exchanges | IPO Dates | Market Lot & Amounts | Investor Quota Split | Anchor Details | Promoter Holdings (Pre/Post) | Financial Performance (FY23–FY25) | Valuation Ratios (EPS, ROE, ROCE, D/E, NAV) | Lead Managers & Registrar | Company Overview | Peer Comparison |

Rules:
- If a value is missing, leave it blank.
- Use only the data present in the HTML.
- Keep it concise.
- Output only the Markdown table, nothing else.

### HTML for {ipo_name}:
{html}
"""


def call_perplexity(prompt: str) -> str:
    """Send the HTML to Perplexity API."""
    headers = {
        "Authorization": f"Bearer {PPLX_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL_NAME,
        "temperature": 0.1,
        "max_tokens": 2000,
        "messages": [
            {"role": "system", "content": "You are a precise IPO data extractor."},
            {"role": "user", "content": prompt}
        ]
    }
    try:
        resp = requests.post(PPLX_API_URL, headers=headers, json=payload, timeout=60)
        resp.raise_for_status()
        data = resp.json()
        return data.get("choices", [{}])[0].get("message", {}).get("content", "")
    except Exception as e:
        return f"⚠️ API error: {e}"


def extract_table(text: str) -> str:
    """Extract Markdown table."""
    match = re.search(r"(\|.+\|[\s\S]*)", text)
    return match.group(1).strip() if match else ""


def parse_markdown_table(md_text: str) -> dict:
    """Convert one-row markdown table into dict."""
    lines = [ln.strip() for ln in md_text.splitlines() if ln.strip()]
    if len(lines) < 3:
        return {}
    headers = [h.strip() for h in lines[0].strip("|").split("|")]
    values = [v.strip() for v in lines[2].strip("|").split("|")]
    return dict(zip(headers, values))
# ------------------------------------------------------


# ---------- MAIN PROCESS ----------
def process_ipo(name: str):
    """Fetch HTML, call Perplexity, store in Mongo."""
    slug = make_two_word_slug(name)
    url = f"https://ipowatch.in/{slug}-ipo-date-review-price-allotment-details/"
    print(f"\n🔍 Processing: {name}\n🌐 {url}")

    html = fetch_html(url)
    if "⚠️ Fetch failed" in html:
        return {"ipo": name, "status": "fetch_failed", "url": url}

    prompt = generate_prompt(name, html)
    result = call_perplexity(prompt)

    if "⚠️ API error" in result or not result:
        return {"ipo": name, "status": "api_failed", "url": url}

    table_md = extract_table(result)
    if not table_md:
        return {"ipo": name, "status": "no_table", "url": url, "raw": result[:300]}

    parsed = parse_markdown_table(table_md)
    parsed["ipo"] = name
    parsed["url"] = url
    parsed["raw_markdown"] = table_md
    parsed["extracted_at"] = datetime.utcnow().isoformat()

    # Save to Mongo
    try:
        client = MongoClient(MONGO_URI)
        coll = client[DB_NAME][COL_IPOS]
        coll.update_one({"ipo": name}, {"$set": {"extracted_fields": parsed}}, upsert=True)
        client.close()
    except Exception as e:
        return {"ipo": name, "status": f"mongo_error: {e}"}

    print(f"✅ Extracted & saved for {name}")
    return {"ipo": name, "status": "ok", "url": url}


def get_unprocessed_ipos(limit=2):
    """Fetch IPO names from MongoDB where extracted_fields missing."""
    client = MongoClient(MONGO_URI)
    coll = client[DB_NAME][COL_IPOS]
    ipos = coll.find({"extracted_fields": {"$exists": False}}, {"ipo": 1}).limit(limit)
    names = [x["ipo"] for x in ipos if "ipo" in x]
    client.close()
    return names
# ------------------------------------------------------


# ---------- ENTRY POINT ----------
if __name__ == "__main__":
    ipo_names = get_unprocessed_ipos(limit=MAX_WORKERS)
    if not ipo_names:
        print("✅ All IPOs already processed.")
        exit()

    print(f"📡 Processing IPOs in parallel: {ipo_names}")

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(process_ipo, name): name for name in ipo_names}
        for fut in as_completed(futures):
            res = fut.result()
            print(f"➡️ {res['ipo']}: {res['status']}")

    print("\n🏁 Done — extracted info for up to 2 IPOs in parallel.")


📡 Processing IPOs in parallel: ['PhysicsWallah', 'Emmvee Photovoltaic Power']

🔍 Processing: PhysicsWallah
🌐 https://ipowatch.in/physicswallah-ipo-date-review-price-allotment-details/

🔍 Processing: Emmvee Photovoltaic Power
🌐 https://ipowatch.in/emmvee-photovoltaic-ipo-date-review-price-allotment-details/
✅ Extracted & saved for Emmvee Photovoltaic Power
➡️ Emmvee Photovoltaic Power: ok
✅ Extracted & saved for PhysicsWallah
➡️ PhysicsWallah: ok

🏁 Done — extracted info for up to 2 IPOs in parallel.


In [ ]:
import os
import re
import json
import requests
from datetime import datetime
from pymongo import MongoClient
from pyspark import SparkConf, SparkContext
import findspark

# ---------- ENVIRONMENT ----------
findspark.init()

python_path = r"C:\Users\prana\AppData\Local\Programs\Python\Python311\python.exe"
os.environ["PYSPARK_PYTHON"] = python_path
os.environ["PYSPARK_DRIVER_PYTHON"] = python_path
os.environ["SPARK_LOCAL_DIRS"] = "C:/spark-temp"
os.makedirs("C:/spark-temp", exist_ok=True)

# Stop any old context
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"

PPLX_API_KEY = os.getenv("PPLX_API_KEY", "your pplx key")
PPLX_API_URL = "https://api.perplexity.ai/chat/completions"
MODEL_NAME = "sonar-pro"

PROCESS_N = 2   # only 2 IPOs parallel for now

# ---------- HELPER FUNCTIONS ----------
def make_two_word_slug(name: str) -> str:
    tokens = re.findall(r"[A-Za-z0-9]+", name)
    return "-".join(t.lower() for t in tokens[:2]) if tokens else ""

def fetch_html(url: str, limit: int = 70000) -> str:
    try:
        resp = requests.get(url, timeout=25, headers={"User-Agent": "Mozilla/5.0"})
        resp.raise_for_status()
        html = re.sub(r"\s+", " ", resp.text)
        return html[:limit]
    except Exception as e:
        return f"⚠️ Fetch failed: {e}"

def generate_prompt(ipo_name: str, html: str) -> str:
    return f"""
You are a financial data extraction model.

Extract the following 13 fields from the IPO details page HTML below.
Output one Markdown table with these exact columns:

| IPO | Price Band | Issue Size | Issue Type | Listing Exchanges | IPO Dates | Market Lot & Amounts | Investor Quota Split | Anchor Details | Promoter Holdings (Pre/Post) | Financial Performance (FY23–FY25) | Valuation Ratios (EPS, ROE, ROCE, D/E, NAV) | Lead Managers & Registrar | Company Overview | Peer Comparison |

Rules:
- If a value is missing, leave it blank.
- Output only the Markdown table, nothing else.

### HTML for {ipo_name}:
{html}
"""

def call_perplexity(prompt: str) -> str:
    headers = {
        "Authorization": f"Bearer {PPLX_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL_NAME,
        "temperature": 0.1,
        "max_tokens": 2000,
        "messages": [
            {"role": "system", "content": "You are a precise IPO data extractor."},
            {"role": "user", "content": prompt}
        ]
    }
    try:
        resp = requests.post(PPLX_API_URL, headers=headers, json=payload, timeout=60)
        resp.raise_for_status()
        data = resp.json()
        return data.get("choices", [{}])[0].get("message", {}).get("content", "")
    except Exception as e:
        return f"⚠️ API error: {e}"

def extract_table(text: str) -> str:
    match = re.search(r"(\|.+\|[\s\S]*)", text)
    return match.group(1).strip() if match else ""

def parse_markdown_table(md_text: str) -> dict:
    lines = [ln.strip() for ln in md_text.splitlines() if ln.strip()]
    if len(lines) < 3:
        return {}
    headers = [h.strip() for h in lines[0].strip("|").split("|")]
    values = [v.strip() for v in lines[2].strip("|").split("|")]
    return dict(zip(headers, values))

# ---------- MAIN TASK ----------
def process_ipo(name: str):
    slug = make_two_word_slug(name)
    url = f"https://ipowatch.in/{slug}-ipo-date-review-price-allotment-details/"
    print(f"\n🔍 Processing: {name}\n🌐 {url}")

    html = fetch_html(url)
    if "⚠️ Fetch failed" in html:
        return {"ipo": name, "status": "fetch_failed", "url": url}

    prompt = generate_prompt(name, html)
    result = call_perplexity(prompt)

    if "⚠️ API error" in result or not result:
        return {"ipo": name, "status": "api_failed", "url": url}

    table_md = extract_table(result)
    if not table_md:
        return {"ipo": name, "status": "no_table", "url": url, "raw": result[:300]}

    parsed = parse_markdown_table(table_md)
    parsed["ipo"] = name
    parsed["url"] = url
    parsed["raw_markdown"] = table_md
    parsed["extracted_at"] = datetime.utcnow().isoformat()

    try:
        client = MongoClient(MONGO_URI)
        coll = client[DB_NAME][COL_IPOS]
        coll.update_one({"ipo": name}, {"$set": {"extracted_fields": parsed}}, upsert=True)
        client.close()
    except Exception as e:
        return {"ipo": name, "status": f"mongo_error: {e}"}

    print(f"✅ Extracted & saved for {name}")
    return {"ipo": name, "status": "ok"}

# ---------- MONGO FETCH ----------
def get_unprocessed_ipos(limit=2):
    client = MongoClient(MONGO_URI)
    coll = client[DB_NAME][COL_IPOS]
    ipos = coll.find({"extracted_fields": {"$exists": False}}, {"ipo": 1}).limit(limit)
    names = [x["ipo"] for x in ipos if "ipo" in x]
    client.close()
    return names

# ---------- SPARK SETUP ----------
conf = (
    SparkConf()
    .setMaster("local[2]")
    .setAppName("IPO-Parallel-RDD")
    .set("spark.python.worker.reuse", "false")
    .set("spark.local.dir", "C:/spark-temp")
)
sc = SparkContext(conf=conf)

# ---------- DRIVER ----------
if __name__ == "__main__":
    ipo_names = get_unprocessed_ipos(limit=PROCESS_N)
    if not ipo_names:
        print("✅ All IPOs already processed.")
        sc.stop()
        exit()

    print(f"📡 Running Spark parallel extraction for: {ipo_names}")
    rdd = sc.parallelize(ipo_names, len(ipo_names))

    results = rdd.map(process_ipo).collect()

    print("\n📊 Extraction Summary:")
    for res in results:
        print(f"➡️ {res['ipo']}: {res['status']}")

    sc.stop()
    print("\n🏁 SparkContext stopped — Done.")


📡 Running Spark parallel extraction for: ['Shining Tools', 'Curis Lifesciences']

📊 Extraction Summary:
➡️ Shining Tools: ok
➡️ Curis Lifesciences: ok

🏁 SparkContext stopped — Done.


In [ ]:
import os
import re
import time
import requests
from datetime import datetime
from pymongo import MongoClient
from pyspark import SparkConf, SparkContext
import findspark
from typing import Optional

# ---------- ENV ----------
findspark.init()
python_path = r"C:\Users\prana\AppData\Local\Programs\Python\Python311\python.exe"
os.environ["PYSPARK_PYTHON"] = python_path
os.environ["PYSPARK_DRIVER_PYTHON"] = python_path
os.environ["SPARK_LOCAL_DIRS"] = "C:/spark-temp"
os.makedirs("C:/spark-temp", exist_ok=True)

if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"

PPLX_API_KEY = os.getenv("PPLX_API_KEY", "your pplx key")
PPLX_API_URL = "https://api.perplexity.ai/chat/completions"
MODEL_NAME = "sonar-pro"

SPARK_PARALLELISM = int(os.getenv("SPARK_PARALLELISM", "4"))
API_DELAY = float(os.getenv("API_DELAY", "1.5"))  # throttle to avoid rate limits
FETCH_RETRIES = 3
FETCH_DELAY = 2  # seconds backoff

# Which fields must be present (your 15 fields + metadata)
REQUIRED_FIELDS = [
    "Price Band", "Issue Size", "Issue Type", "Listing Exchanges", "IPO Dates",
    "Market Lot & Amounts", "Investor Quota Split", "Anchor Details",
    "Promoter Holdings (Pre/Post)", "Financial Performance (FY23–FY25)",
    "Valuation Ratios (EPS, ROE, ROCE, D/E, NAV)", "Lead Managers & Registrar",
    "Company Overview", "Peer Comparison", "ipo"
]

# ---------- HELPERS ----------
def make_slug_candidates(name: str):
    """Return multiple slug variants to try on IPOWatch."""
    toks = re.findall(r"[A-Za-z0-9]+", name.lower())
    candidates = []
    if toks:
        # first two words
        candidates.append("-".join(toks[:2]))
        # first three words
        if len(toks) >= 3:
            candidates.append("-".join(toks[:3]))
        # entire name as slug (trim to first 6)
        candidates.append("-".join(toks[:6]))
    # add safe fallback: name with hyphens
    candidates.append("-".join(toks))
    return list(dict.fromkeys([c for c in candidates if c]))  # unique preserve order

def fetch_html_try(url: str, retries=FETCH_RETRIES, delay=FETCH_DELAY) -> Optional[str]:
    headers = {"User-Agent": "Mozilla/5.0 (compatible; IPOBot/1.0)"}
    for attempt in range(1, retries+1):
        try:
            r = requests.get(url, timeout=25, headers=headers, allow_redirects=True)
            status = r.status_code
            if status == 200 and r.text:
                return re.sub(r"\s+", " ", r.text)[:120000]  # keep top content
            else:
                # non-200 often means not found; don't retry too many times but still backoff a little
                time.sleep(delay * attempt)
        except Exception as e:
            time.sleep(delay * attempt)
    return None

def fetch_ipowatch_html_for(ipo_name: str):
    """Try several URL patterns and site search to get the IPOWatch HTML."""
    base = "https://ipowatch.in"
    # Try candidate slugs
    for slug in make_slug_candidates(ipo_name):
        url1 = f"{base}/{slug}-ipo-date-review-price-allotment-details/"
        html = fetch_html_try(url1)
        if html:
            return html, url1
        # try simple slug
        url2 = f"{base}/{slug}/"
        html = fetch_html_try(url2)
        if html:
            return html, url2
    # fallback: site search (ipowatch uses ?s=)
    try:
        search_url = f"{base}/?s={requests.utils.quote(ipo_name)}"
        html = fetch_html_try(search_url)
        if html:
            return html, search_url
    except:
        pass
    return None, None

def call_perplexity(prompt: str) -> Optional[str]:
    headers = {"Authorization": f"Bearer {PPLX_API_KEY}", "Content-Type": "application/json"}
    payload = {
        "model": MODEL_NAME,
        "temperature": 0.1,
        "max_tokens": 3000,
        "messages": [
            {"role": "system", "content": "You are a precise IPO data extraction assistant. Output only a single-row Markdown table with the requested columns."},
            {"role": "user", "content": prompt}
        ]
    }
    try:
        r = requests.post(PPLX_API_URL, headers=headers, json=payload, timeout=90)
        r.raise_for_status()
        data = r.json()
        return data.get("choices", [{}])[0].get("message", {}).get("content", "")
    except Exception as e:
        return None

def extract_table(text: str) -> Optional[str]:
    if not text:
        return None
    m = re.search(r"(\|.+\|[\s\S]*)", text)
    return m.group(1).strip() if m else None

def parse_markdown_table(md_text: str) -> dict:
    """Parse a single-row markdown table into a dict."""
    lines = [ln.strip() for ln in md_text.splitlines() if ln.strip()]
    if len(lines) < 3:
        return {}
    headers = [h.strip() for h in lines[0].strip("|").split("|")]
    values = [v.strip() for v in lines[2].strip("|").split("|")]
    # pad values if shorter
    if len(values) < len(headers):
        values += [""] * (len(headers) - len(values))
    return dict(zip(headers, values))

def count_present_fields(extracted: dict):
    """Count how many required fields are present and non-empty."""
    if not extracted:
        return 0
    cnt = 0
    for k in REQUIRED_FIELDS:
        if k == "ipo":
            # 'ipo' mandatory as a sanity check
            if extracted.get("ipo"):
                cnt += 1
        else:
            v = extracted.get(k)
            if v is not None and str(v).strip() != "":
                cnt += 1
    return cnt

def mongo_get_all_ipos():
    client = MongoClient(MONGO_URI)
    coll = client[DB_NAME][COL_IPOS]
    docs = list(coll.find({}, {"ipo": 1, "extracted_fields": 1}))
    client.close()
    return docs

def mongo_update_partial(ipo_name: str, new_fields: dict, source_url: str, raw_md: str):
    """Merge new_fields into existing document's 'extracted_fields' and append to history."""
    client = MongoClient(MONGO_URI)
    coll = client[DB_NAME][COL_IPOS]
    doc = coll.find_one({"ipo": ipo_name}) or {}
    existing = doc.get("extracted_fields") or {}
    # Merge: prefer existing non-empty value, otherwise take new
    merged = existing.copy()
    for k, v in new_fields.items():
        # if existing missing or empty, replace
        if (k not in merged) or (merged.get(k) in [None, "", [], {}]):
            merged[k] = v
    # metadata
    history_entry = {
        "extracted_at": datetime.utcnow().isoformat(),
        "source_url": source_url,
        "raw_markdown": raw_md,
        "fields_added": {k: v for k, v in new_fields.items() if (existing.get(k) in [None, "", {}, []])}
    }
    # update doc
    coll.update_one({"ipo": ipo_name}, {"$set": {"extracted_fields": merged, "last_extracted_at": history_entry["extracted_at"]}, "$push": {"extraction_history": history_entry}}, upsert=True)
    client.close()

# ---------- PROCESS ONE IPO ----------
def process_ipo(name: str):
    print(f"\n🔍 Starting: {name}")
    # 1) fetch HTML (try many patterns)
    html, url = fetch_ipowatch_html_for(name)
    if not html:
        print(f"⚠️ Fetch failed for {name} (tried multiple slugs/search).")
        return {"ipo": name, "status": "fetch_failed"}

    # 2) load existing extracted_fields to include in prompt (so model only fills missing)
    client = MongoClient(MONGO_URI)
    coll = client[DB_NAME][COL_IPOS]
    doc = coll.find_one({"ipo": name}) or {}
    client.close()
    existing = doc.get("extracted_fields") or {}

    # 3) Build a focused prompt: include list of missing fields and existing values
    missing = [f for f in REQUIRED_FIELDS if not existing.get(f)]
    # always include 'ipo' field in the table header
    header_cols = REQUIRED_FIELDS.copy()

    prompt_lines = [
        "You are an expert IPO data extractor. You will be given an HTML snippet (IPOWatch) and a list of existing extracted fields (may be partial).",
        f"IPO name: {name}",
        "",
        "Existing extracted fields (only show values already present):"
    ]
    if existing:
        for k, v in existing.items():
            prompt_lines.append(f"- {k}: {v}")
    else:
        prompt_lines.append("- (none)")

    prompt_lines += [
        "",
        f"Please extract the following missing fields (fill blanks). Missing fields: {missing}",
        "Output a single-row Markdown table with this exact header (in this order):",
        "| " + " | ".join(header_cols) + " |",
        "",
        "Rules:",
        "- If a value already exists in 'Existing extracted fields', preserve it in the output.",
        "- Only output the table (no commentary).",
        "- Leave values blank where you cannot find them.",
        "",
        "HTML:",
        html[:90000]  # keep prompt bounded
    ]

    prompt = "\n".join(prompt_lines)

    # 4) call Perplexity
    result_text = call_perplexity(prompt)
    time.sleep(API_DELAY)
    if not result_text:
        print(f"⚠️ API failed for {name}")
        return {"ipo": name, "status": "api_failed"}

    table_md = extract_table(result_text)
    if not table_md:
        print(f"⚠️ No table parsed from LLM output for {name}. Raw start:\n{result_text[:400]}")
        return {"ipo": name, "status": "no_table", "raw_snippet": result_text[:400]}

    parsed = parse_markdown_table(table_md)
    # ensure 'ipo' key present
    parsed["ipo"] = parsed.get("ipo", name)
    # merge into mongo
    mongo_update_partial(name, parsed, url, table_md)
    print(f"✅ Updated {name} (added {len(parsed)} fields, merged).")
    return {"ipo": name, "status": "ok"}

# ---------- DRIVER ----------
conf = (
    SparkConf()
    .setMaster(f"local[{SPARK_PARALLELISM}]")
    .setAppName("IPO-Fill-Missing-Fields")
    .set("spark.python.worker.reuse", "false")
    .set("spark.local.dir", "C:/spark-temp")
)
sc = SparkContext(conf=conf)

if __name__ == "__main__":
    # fetch all IPO docs and compute completeness
    docs = mongo_get_all_ipos()
    candidates = []
    for d in docs:
        name = d.get("ipo")
        existing = d.get("extracted_fields") or {}
        present = count_present_fields(existing)
        completeness = present / len(REQUIRED_FIELDS)
        if completeness < 1.0:
            candidates.append((name, present, completeness))
    # sort by least-complete first
    candidates = sorted(candidates, key=lambda x: (x[2], -x[1]))
    ipo_names = [c[0] for c in candidates]

    if not ipo_names:
        print("✅ All IPOs already complete (100%). Nothing to do.")
        sc.stop()
        exit()

    print(f"📡 Need to re-extract/enrich {len(ipo_names)} IPOs — running with {SPARK_PARALLELISM} workers...")
    rdd = sc.parallelize(ipo_names, min(len(ipo_names), SPARK_PARALLELISM))
    results = rdd.map(process_ipo).collect()

    print("\n📊 Extraction Summary:")
    for res in results:
        print(f"➡️ {res['ipo']}: {res['status']}")

    sc.stop()
    print("\n🏁 Done — Spark stopped.")


📡 Need to re-extract/enrich 10 IPOs — running with 4 workers...

📊 Extraction Summary:
➡️ Finbud Financial: ok
➡️ Shreeji Global FMCG: ok
➡️ Tenneco Clean Air India: ok
➡️ Mahamaya Lifesciences: ok
➡️ Tenneco Clean Air: ok
➡️ Curis Lifesciences: ok
➡️ PhysicsWallah: ok
➡️ Emmvee Photovoltaic Power: ok
➡️ Shining Tools: ok
➡️ Workmates Core2Cloud: ok

🏁 Done — Spark stopped.


In [3]:
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

# MongoDB connection
MONGO_URI = "mongodb://localhost:27017"
DB_NAME = "ipo_db"
COL_IPOS = "ipos"

def analyze_mongo_data():
    # Connect to MongoDB
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COL_IPOS]
    
    # Get all IPO records
    records = list(collection.find())
    print(f"\n🔍 Found {len(records)} total IPO records\n")

    # Analyze data completeness
    fields_present = {}
    extracted_fields_present = {}
    
    for record in records:
        # Check top-level fields
        for field in record.keys():
            fields_present[field] = fields_present.get(field, 0) + 1
            
        # Check extracted_fields if present
        if 'extracted_fields' in record:
            extracted = record['extracted_fields']
            for field in extracted.keys():
                extracted_fields_present[field] = extracted_fields_present.get(field, 0) + 1
    
    print("📊 Top-level fields presence:")
    for field, count in fields_present.items():
        print(f"{field}: {count}/{len(records)} records")
    
    print("\n📊 Extracted fields presence:")
    for field, count in extracted_fields_present.items():
        print(f"{field}: {count}/{len(records)} records")
    
    # Print a sample record with extracted fields
    sample = collection.find_one({"extracted_fields": {"$exists": True}})
    if sample:
        print("\n📝 Sample IPO record with extracted fields:")
        pprint(sample)

    client.close()

if __name__ == "__main__":
    analyze_mongo_data()


🔍 Found 10 total IPO records

📊 Top-level fields presence:
_id: 10/10 records
ipo: 10/10 records
category: 10/10 records
gmp_investorgain: 10/10 records
gmp_ipowatch: 10/10 records
gmp_diff: 10/10 records
issue_price: 10/10 records
open_date: 10/10 records
close_date: 10/10 records
raw: 10/10 records
inserted_at: 10/10 records
extracted_fields: 10/10 records
extraction_history: 10/10 records
last_extracted_at: 10/10 records

📊 Extracted fields presence:
IPO: 4/10 records
Price Band: 8/10 records
Issue Size: 8/10 records
Issue Type: 8/10 records
Listing Exchanges: 8/10 records
IPO Dates: 8/10 records
Market Lot & Amounts: 8/10 records
Investor Quota Split: 8/10 records
Anchor Details: 8/10 records
Promoter Holdings (Pre/Post): 8/10 records
Financial Performance (FY23–FY25): 8/10 records
Valuation Ratios (EPS, ROE, ROCE, D/E, NAV): 8/10 records
Lead Managers & Registrar: 8/10 records
Company Overview: 8/10 records
Peer Comparison: 8/10 records
ipo: 10/10 records
url: 8/10 records
raw_ma

In [2]:
import os
import re
import math
import requests
from datetime import datetime
from pymongo import MongoClient
from pyspark import SparkConf, SparkContext
import findspark

# ---------- SPARK & ENV SETUP ----------
findspark.init()
PYTHON_PATH = r"C:\Users\prana\AppData\Local\Programs\Python\Python311\python.exe"
os.environ["PYSPARK_PYTHON"] = PYTHON_PATH
os.environ["PYSPARK_DRIVER_PYTHON"] = PYTHON_PATH
os.environ["SPARK_LOCAL_DIRS"] = "C:/spark-temp"
os.makedirs("C:/spark-temp", exist_ok=True)

if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_ANALYSIS = "ipo_analysis"

SPARK_PARALLELISM = 4

# ---------- SCORING WEIGHTS ----------
W_GMP = 0.45
W_PRICE = 0.20
W_SIZE = 0.20
W_EXPECT = 0.15

# ---------- UTILITIES ----------
def get_mongo_data():
    client = MongoClient(MONGO_URI)
    data = list(client[DB_NAME][COL_IPOS].find({}))
    client.close()
    return data

def write_results(results):
    client = MongoClient(MONGO_URI)
    coll = client[DB_NAME][COL_ANALYSIS]
    for rec in results:
        coll.update_one({"ipo": rec["ipo"]}, {"$set": rec}, upsert=True)
    client.close()

def safe_float(x):
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except:
        return None

def parse_issue_price(value):
    """Parse avg issue price"""
    if isinstance(value, dict):
        return value.get("avg") or value.get("mid") or value.get("min")
    if not value:
        return None
    nums = re.findall(r"\d+\.?\d*", str(value))
    if not nums:
        return None
    nums = [safe_float(x) for x in nums]
    return sum(nums)/len(nums)

def parse_issue_size(value):
    if not value:
        return None
    s = str(value).lower().replace(",", "")
    m = re.search(r"(\d+\.?\d*)\s*(cr|crore|lakh|lac|mn|m|bn|b)?", s)
    if not m:
        return None
    num = float(m.group(1))
    mult = (m.group(2) or "").lower()
    if mult in ["cr", "crore"]:
        return num * 1e7
    if mult in ["lakh", "lac"]:
        return num * 1e5
    if mult in ["mn", "m"]:
        return num * 1e6
    if mult in ["bn", "b"]:
        return num * 1e9
    return num

# ---------- SCORING ----------
def compute_score(doc):
    fields = doc.get("extracted_fields", {}) or {}
    ipo = doc.get("ipo") or fields.get("IPO")

    # Skip IPOs with too few extracted fields
    nonempty_fields = len([v for v in fields.values() if v not in [None, "", {}]])
    if nonempty_fields < 5:
        return {"ipo": ipo, "status": f"skip_few_fields({nonempty_fields})"}

    # Try multiple fallbacks for GMP
    gmp = safe_float(fields.get("GMP_InvestorGain")) or safe_float(fields.get("GMP_IPOWatch")) \
        or safe_float(doc.get("gmp_investorgain")) or safe_float(doc.get("gmp_ipowatch"))
    issue_price = parse_issue_price(fields.get("Price Band") or fields.get("Issue_Price") or doc.get("issue_price"))
    issue_size = parse_issue_size(fields.get("Issue Size") or fields.get("Issue_Size") or doc.get("issue_size"))

    if not all([gmp, issue_price, issue_size]):
        return {"ipo": ipo, "status": "missing_critical_data"}

    # 1️⃣ GMP Score (relative to issue price)
    gmp_pct = (gmp / issue_price) * 100
    gmp_score = min(max(gmp_pct, 0), 100)

    # 2️⃣ Price Score (retail affordability)
    if issue_price < 100:
        price_score = 90
    elif issue_price < 500:
        price_score = 80
    elif issue_price < 1000:
        price_score = 60
    else:
        price_score = 40

    # 3️⃣ Size Score (mid-size preferred)
    if issue_size < 1e8:
        size_score = 40
    elif issue_size < 1e9:
        size_score = 70
    elif issue_size < 5e9:
        size_score = 90
    else:
        size_score = 60

    # 4️⃣ Expected Listing Gain
    expect_score = min(max(gmp_pct / 2 + 50, 0), 100)

    # Weighted average
    total = (
        W_GMP * gmp_score +
        W_PRICE * price_score +
        W_SIZE * size_score +
        W_EXPECT * expect_score
    )

    score = round((total / 100) * 9 + 1, 1)
    verdict = "Good" if score >= 7 else "Moderate" if score >= 4 else "Bad"

    return {
        "ipo": ipo,
        "gmp": gmp,
        "issue_price": issue_price,
        "issue_size": issue_size,
        "gmp_pct": round(gmp_pct, 2),
        "score": score,
        "verdict": verdict,
        "scored_at": datetime.utcnow().isoformat(),
        "components": {
            "GMP": gmp_score,
            "Price": price_score,
            "Size": size_score,
            "Expectation": expect_score
        },
        "status": "scored"
    }

# ---------- SPARK SETUP ----------
conf = (
    SparkConf()
    .setMaster(f"local[{SPARK_PARALLELISM}]")
    .setAppName("IPO-Scoring-v2")
    .set("spark.local.dir", "C:/spark-temp")
)
sc = SparkContext(conf=conf)

# ---------- MAIN ----------
if __name__ == "__main__":
    print("📡 Fetching IPO data from MongoDB...")
    docs = get_mongo_data()
    print(f"🔍 Found {len(docs)} total IPO records.")

    if not docs:
        print("⚠️ No records found.")
        sc.stop()
        exit()

    rdd = sc.parallelize(docs, min(len(docs), SPARK_PARALLELISM))
    results = rdd.map(compute_score).collect()
    sc.stop()

    scored = [r for r in results if r.get("status") == "scored"]
    skipped = [r for r in results if r.get("status") != "scored"]

    if scored:
        write_results(scored)
        print(f"✅ {len(scored)} IPOs scored and written to '{COL_ANALYSIS}'")
    else:
        print("⚠️ No IPOs scored (all skipped).")

    print("\n📊 Summary:")
    print("Scored:", len(scored))
    print("Skipped:", len(skipped))
    if skipped:
        print("Skipped details:")
        for s in skipped:
            print(f" - {s['ipo']}: {s['status']}")

    if scored:
        print("\n🏆 Top Recommendations:")
        top = sorted(scored, key=lambda x: x["score"], reverse=True)[:10]
        for t in top:
            print(f" {t['ipo']}: {t['score']}/10 — {t['verdict']} (GMP {t['gmp_pct']}%)")

    print("\n🏁 Done.")


📡 Fetching IPO data from MongoDB...
🔍 Found 10 total IPO records.
✅ 6 IPOs scored and written to 'ipo_analysis'

📊 Summary:
Scored: 6
Skipped: 4
Skipped details:
 - Tenneco Clean Air India: missing_critical_data
 - Mahamaya Lifesciences: skip_few_fields(4)
 - Curis Lifesciences: skip_few_fields(4)
 - Finbud Financial: missing_critical_data

🏆 Top Recommendations:
 PhysicsWallah: 5.8/10 — Moderate (GMP 33.02%)
 Emmvee Photovoltaic Power: 5.1/10 — Moderate (GMP 18.91%)
 Workmates Core2Cloud: 4.9/10 — Moderate (GMP 10.89%)
 Shining Tools: 4.9/10 — Moderate (GMP 10.53%)
 Tenneco Clean Air: 4.7/10 — Moderate (GMP 11.61%)
 Shreeji Global FMCG: 4.7/10 — Moderate (GMP 6.53%)

🏁 Done.


In [3]:
import os
import re
import math
from datetime import datetime
from pymongo import MongoClient

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_ANALYSIS = "ipo_analysis"
COL_RECOMMEND = "ipo_portfolio_recommendations"

MIN_INVEST_MAINBOARD = 15000  # INR minimum investment for mainboard (user requirement)

# ---------- HELPERS ----------
def safe_float(x):
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except:
        return None

def parse_price_mid(issue_price_field):
    """Try to get average/mid issue price from possible field formats."""
    if not issue_price_field:
        return None
    if isinstance(issue_price_field, dict):
        return safe_float(issue_price_field.get("avg") or issue_price_field.get("mid") or issue_price_field.get("min"))
    # string like "₹206-₹217" or "206-217"
    s = str(issue_price_field)
    nums = re.findall(r"\d+\.?\d*", s)
    nums = [safe_float(n) for n in nums if safe_float(n) is not None]
    if not nums:
        return None
    return sum(nums) / len(nums)

def parse_lot_from_str(s):
    """
    Parse market lot from strings like 'Lot Size: 137' or 'Market Lot 69' or 'Lot: 1200 shares'
    Return integer lot if found else None.
    """
    if not s:
        return None
    s = str(s)
    # common patterns: "Lot: 1200", "lot size 1200", "market lot 1200"
    m = re.search(r"(?:lot size|lot|market lot|market_lot)\D*(\d{1,5})", s, flags=re.I)
    if m:
        try:
            return int(m.group(1))
        except:
            return None
    # fallback: find first reasonable integer > 1
    nums = re.findall(r"\d+", s)
    for n in nums:
        val = int(n)
        if val > 1 and val < 100000:
            return val
    return None

def get_mongo_col(colname):
    client = MongoClient(MONGO_URI)
    return client[DB_NAME][colname], client

def load_ipos_and_scores():
    ipos_col, client = get_mongo_col(COL_IPOS)
    analysis_col = client[DB_NAME][COL_ANALYSIS]
    ipos_docs = list(ipos_col.find({}))
    # build dict by ipo name
    ipos_by_name = {}
    for d in ipos_docs:
        name = d.get("ipo")
        ipos_by_name[name] = d
    # load analysis
    scored = {}
    for a in list(analysis_col.find({})):
        name = a.get("ipo")
        if name:
            scored[name] = a
    client.close()
    return ipos_by_name, scored

# ---------- ALLOCATION ALGORITHM ----------
def prepare_candidates(ipos_by_name, scored, hold_until_date_obj):
    """
    Build candidate list with fields:
    - ipo
    - category (Mainboard/SME)
    - score (1..10)
    - issue_mid
    - lot (int or None)
    - min_invest (INR) = lot*issue_mid or MIN_INVEST_MAINBOARD
    - close_date (date object)
    - total_score_per_rupee = score / min_invest (used for greedy)
    """
    candidates = []
    for name, ipo_doc in ipos_by_name.items():
        analysis = scored.get(name)
        if not analysis or analysis.get("status") != "scored":
            continue
        fields = ipo_doc.get("extracted_fields", {}) or {}
        # close_date fallback top-level
        close_date_str = fields.get("Close_Date") or ipo_doc.get("close_date") or fields.get("IPO Dates") or ipo_doc.get("close_date")
        # try to parse close date in formats we might have
        close_date = None
        if close_date_str:
            try:
                # some strings can be ranges like "11-Nov to 13-Nov", so pick first date-like substring
                ds = re.findall(r"\d{4}-\d{2}-\d{2}|\d{1,2}[-/][A-Za-z]{3,9}[-/]\d{2,4}|\d{1,2}\s+[A-Za-z]{3,9}\s+\d{4}", str(close_date_str))
                if ds:
                    close_date = try_parse_date(ds[0])
                else:
                    close_date = try_parse_date(close_date_str)
            except:
                close_date = None
        # if close_date missing, skip (we need it for horizon filter)
        if not close_date:
            continue
        # only include if close_date <= hold_until_date_obj (user can hold until)
        if close_date > hold_until_date_obj:
            # NOT eligible — user cannot hold until listing/allotment
            continue

        # get category
        category = ipo_doc.get("category") or fields.get("Category") or fields.get("IPO Type") or ""
        category = category.strip().lower()

        # score
        score = analysis.get("score")
        if score is None:
            continue

        # issue price mid
        issue_mid = parse_price_from_fields(fields, ipo_doc)

        if issue_mid is None:
            continue

        # lot parsing
        lot = None
        lot = parse_lot_from_str(fields.get("Market Lot & Amounts") or fields.get("Market Lot") or fields.get("Market_Lot") or fields.get("Lot") or ipo_doc.get("lot"))
        # fallback: sometimes 'Market Lot & Amounts' contains numbers; try parse
        if not lot:
            lot = parse_lot_from_str(fields.get("Market Lot & Amounts") or "")

        # compute min_invest
        if "sme" in category:
            # for SME we need a lot to compute min investment; if not found, skip SME
            if not lot:
                continue
            min_invest = lot * issue_mid
        else:
            # mainboard: if lot present use lot*issue_mid but ensure min at least MIN_INVEST_MAINBOARD
            if lot:
                min_invest = max(MIN_INVEST_MAINBOARD, lot * issue_mid)
            else:
                min_invest = MIN_INVEST_MAINBOARD

        # prepare candidate
        candidates.append({
            "ipo": name,
            "category": "SME" if "sme" in category else "Mainboard",
            "score": score,
            "issue_mid": issue_mid,
            "lot": lot,
            "min_invest": float(min_invest),
            "close_date": close_date,
            "gmp_pct": analysis.get("gmp_pct")
        })
    return candidates

def try_parse_date(s):
    """Try several date formats and return date object; fallback to None"""
    s = str(s).strip()
    # try ISO first
    try:
        return datetime.strptime(s[:10], "%Y-%m-%d").date()
    except:
        pass
    # try common formats
    fmts = ["%d-%b-%Y", "%d-%b-%y", "%d %b %Y", "%d %b %y", "%d-%m-%Y", "%d/%m/%Y"]
    for f in fmts:
        try:
            return datetime.strptime(s, f).date()
        except:
            pass
    # try parse by splitting and using day-month combos
    try:
        import dateutil.parser
        return dateutil.parser.parse(s, dayfirst=False).date()
    except:
        return None

def parse_price_from_fields(fields, ipo_doc):
    # try several possibilities to get issue mid price
    for key in ["Issue_Price", "Issue Price", "Price Band", "issue_price", "Issue_Price_struct", "issue_price_struct"]:
        v = fields.get(key) or ipo_doc.get(key) or ipo_doc.get("issue_price")
        if v:
            mid = None
            if isinstance(v, dict):
                mid = v.get("avg") or v.get("mid") or v.get("min")
                if mid: return safe_float(mid)
            else:
                # parse numbers
                nums = re.findall(r"\d+\.?\d*", str(v))
                if nums:
                    nums = [safe_float(n) for n in nums if safe_float(n) is not None]
                    if nums:
                        return sum(nums)/len(nums)
    # fallback top-level fields that your pipeline created
    ip = ipo_doc.get("issue_price")
    if ip:
        try:
            return float(ip)
        except:
            pass
    return None

def allocate_budget_greedy(candidates, budget):
    """
    Greedy algorithm:
    - Sort candidates by score per rupee = score / min_invest descending
    - For each candidate, allocate as many minimum "lots" as possible while budget allows,
      but we prefer to allocate only one min_invest per candidate first (diversification),
      then second pass add extra lots proportional to score per rupee.
    - Returns allocation list with lots_count and invested amount.
    """
    # compute score_per_rupee
    for c in candidates:
        c["score_per_rupee"] = (c["score"] / c["min_invest"]) if c["min_invest"] > 0 else 0

    # sort descending by score_per_rupee
    candidates = sorted(candidates, key=lambda x: x["score_per_rupee"], reverse=True)

    allocation = []
    remain = budget

    # first pass: try to give each candidate one minimum unit (diversify)
    for c in candidates:
        if remain >= c["min_invest"]:
            # number of lots to allocate initially = 1 (or for SME maybe 1 lot)
            lots = 1
            invested = c["min_invest"] * lots
            allocation.append({
                "ipo": c["ipo"],
                "category": c["category"],
                "score": c["score"],
                "issue_mid": c["issue_mid"],
                "lot": c["lot"],
                "lots_allocated": lots,
                "invested": invested,
                "min_invest": c["min_invest"],
                "score_per_rupee": c["score_per_rupee"]
            })
            remain -= invested
        else:
            # cannot afford min for this candidate
            allocation.append({
                "ipo": c["ipo"],
                "category": c["category"],
                "score": c["score"],
                "issue_mid": c["issue_mid"],
                "lot": c["lot"],
                "lots_allocated": 0,
                "invested": 0.0,
                "min_invest": c["min_invest"],
                "score_per_rupee": c["score_per_rupee"]
            })

    # second pass: distribute remaining funds greedily into highest score_per_rupee candidates
    # allow adding additional lots for the same IPO (respect multiples)
    changed = True
    while changed and remain >= min([c["min_invest"] for c in candidates]):
        changed = False
        # iterate over sorted candidates
        for idx, c in enumerate(candidates):
            # find allocation entry
            alloc = next((a for a in allocation if a["ipo"] == c["ipo"]), None)
            if not alloc:
                continue
            # cost for one more lot
            cost = c["min_invest"]
            if remain >= cost:
                # add one more lot
                alloc["lots_allocated"] += 1
                alloc["invested"] += cost
                remain -= cost
                changed = True
            # if remain smaller, continue to next candidate
    # remove those with zero lots
    allocation = [a for a in allocation if a["lots_allocated"] > 0]
    return allocation, remain

# ---------- MAIN INTERACTION & RUN ----------
def main():
    print("IPO portfolio optimizer\n")
    # user inputs
    try:
        budget_in = input("Enter total investable amount in INR (e.g., 50000): ").strip()
        budget = float(budget_in)
    except:
        print("Invalid budget. Exiting.")
        return

    hold_until = input("Enter the date until which you can keep money (YYYY-MM-DD): ").strip()
    try:
        hold_date = datetime.strptime(hold_until, "%Y-%m-%d").date()
    except:
        print("Invalid date format. Use YYYY-MM-DD. Exiting.")
        return

    # load data
    ipos_by_name, scored = load_ipos_and_scores()
    print("\nLoaded {} IPOs and {} scored analyses.".format(len(ipos_by_name), len(scored)))

    # prepare candidates
    candidates = prepare_candidates(ipos_by_name, scored, hold_date)
    if not candidates:
        print("No eligible IPOs found for your hold date or missing critical fields. Exiting.")
        return

    print(f"\nFound {len(candidates)} eligible IPO candidates for allocation.\n")
    # show brief list
    for c in candidates:
        lot_text = str(c['lot']) if c['lot'] else "N/A"
        print(f"{c['ipo']} | {c['category']} | Score: {c['score']} | Issue: {c['issue_mid']} | Lot: {lot_text} | Min invest: {int(c['min_invest'])} | Close: {c['close_date']}")

    # allocate
    allocation, leftover = allocate_budget_greedy(candidates, budget)

    # print results
    print("\n--- Allocation Plan ---")
    total_invested = 0.0
    if not allocation:
        print("Could not allocate budget (insufficient funds for minimums).")
    else:
        for a in allocation:
            total_invested += a["invested"]
            print(f"{a['ipo']}: lots={a['lots_allocated']}, invest=₹{int(a['invested'])}, score={a['score']}, issue_mid={a['issue_mid']}, min_invest_unit=₹{int(a['min_invest'])}")
    print(f"\nTotal invested: ₹{int(total_invested)}")
    print(f"Leftover cash: ₹{int(leftover)}")

    # Save recommendation to Mongo
    client = MongoClient(MONGO_URI)
    coll = client[DB_NAME][COL_RECOMMEND]
    rec = {
        "created_at": datetime.utcnow().isoformat(),
        "budget": budget,
        "hold_until": hold_date.isoformat(),
        "allocation": allocation,
        "total_invested": total_invested,
        "leftover": leftover
    }
    coll.insert_one(rec)
    client.close()
    print("\nRecommendation saved to MongoDB collection:", COL_RECOMMEND)
    print("Done.")

if __name__ == "__main__":
    main()


IPO portfolio optimizer


Loaded 10 IPOs and 6 scored analyses.

Found 6 eligible IPO candidates for allocation.

PhysicsWallah | Mainboard | Score: 5.8 | Issue: 103.0 | Lot: 137 | Min invest: 15000 | Close: 2025-11-13
Emmvee Photovoltaic Power | Mainboard | Score: 5.1 | Issue: 206.0 | Lot: 69 | Min invest: 15000 | Close: 2025-11-13
Workmates Core2Cloud | SME | Score: 4.9 | Issue: 200.0 | Lot: 1 | Min invest: 200 | Close: 2025-11-13
Tenneco Clean Air | Mainboard | Score: 4.7 | Issue: 378.0 | Lot: 37 | Min invest: 15000 | Close: 2025-11-14
Shining Tools | SME | Score: 4.9 | Issue: 114.0 | Lot: 2 | Min invest: 228 | Close: 2025-11-11
Shreeji Global FMCG | SME | Score: 4.7 | Issue: 120.0 | Lot: 2 | Min invest: 240 | Close: 2025-11-07

--- Allocation Plan ---
Workmates Core2Cloud: lots=23, invest=₹4600, score=4.9, issue_mid=200.0, min_invest_unit=₹200
Shining Tools: lots=22, invest=₹5016, score=4.9, issue_mid=114.0, min_invest_unit=₹228
Shreeji Global FMCG: lots=22, invest=₹5280, score=4.7

In [1]:
import os
import json
from pymongo import MongoClient
from bson import json_util

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"

# ---------- CONNECT & FETCH ----------
def print_one_ipo_sample():
    try:
        client = MongoClient(MONGO_URI)
        db = client[DB_NAME]
        col = db[COL_IPOS]

        # Fetch one IPO record
        doc = col.find_one()

        if not doc:
            print("⚠️ No records found in collection:", COL_IPOS)
            return

        # Pretty-print JSON
        print("✅ Sample IPO record from MongoDB ('ipo_db.ipos'):\n")
        print(json.dumps(doc, indent=4, default=json_util.default))

    except Exception as e:
        print("❌ Error:", e)

    finally:
        client.close()

# ---------- MAIN ----------
if __name__ == "__main__":
    print_one_ipo_sample()


✅ Sample IPO record from MongoDB ('ipo_db.ipos'):

{
    "_id": {
        "$oid": "690f5be67ebcab56b4d5097a"
    },
    "ipo": "PhysicsWallah",
    "category": "Mainboard",
    "gmp_investorgain": 35.0,
    "gmp_ipowatch": 30.0,
    "gmp_diff": 5.0,
    "issue_price": {
        "min": 103.0,
        "max": 103.0,
        "avg": 103.0
    },
    "open_date": "2025-11-11",
    "close_date": "2025-11-13",
    "raw": {
        "GMP_InvestorGain": "35",
        "GMP_IPOWatch": "30",
        "Issue_Price": "\u20b9103 \u2013 \u20b9109",
        "GMP_Diff": "5"
    },
    "inserted_at": "2025-11-08T15:05:12.990572",
    "extracted_fields": {
        "IPO": "PhysicsWallah Ltd.",
        "Price Band": "\u20b9103 to \u20b9109",
        "Issue Size": "\u20b93,480 Crores",
        "Issue Type": "Book Built Issue",
        "Listing Exchanges": "BSE, NSE",
        "IPO Dates": "Nov 11\u201313, 2025 (Listing: Nov 18, 2025)",
        "Market Lot & Amounts": "Min: 137 shares/\u20b914,933; Max (Retail): 

In [2]:
import os
import re
from datetime import datetime
from pymongo import MongoClient

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_ANALYSIS = "ipo_analysis"
COL_RECOMMEND = "ipo_portfolio_recommendations"

MIN_INVEST_MAINBOARD = 15000  # Default min for mainboard
RETAIL_ONLY = True  # Always retail investor


# ---------- HELPERS ----------
def safe_float(x):
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except Exception:
        return None


def try_parse_date(s):
    if not s:
        return None
    s = str(s).strip()
    # Try YYYY-MM-DD
    try:
        return datetime.strptime(s[:10], "%Y-%m-%d").date()
    except Exception:
        pass
    fmts = ["%d-%b-%Y", "%d %b %Y", "%d/%m/%Y", "%d-%m-%Y", "%d %B %Y"]
    for f in fmts:
        try:
            return datetime.strptime(s, f).date()
        except Exception:
            continue
    try:
        import dateutil.parser
        return dateutil.parser.parse(s, dayfirst=True).date()
    except Exception:
        return None


def parse_lot_and_min_invest(text):
    """
    Parse lot size and min invest from strings like:
    "Min: 137 shares/₹14,933; Max (Retail): 1,781 shares/₹1,94,129"
    Returns (lot_size, min_invest)
    """
    if not text:
        return None, None
    text = str(text)
    # Find first "Min: ..." pattern
    lot = None
    min_inv = None

    # Pattern for something like '137 shares/₹14,933'
    m = re.search(r"(\d{1,5})\s*shares?.*?₹\s?([\d,]+)", text)
    if m:
        lot = safe_float(m.group(1))
        min_inv = safe_float(m.group(2))
    else:
        # try fallback: just ₹ number
        m2 = re.search(r"₹\s?([\d,]+)", text)
        if m2:
            min_inv = safe_float(m2.group(1))

    return lot, min_inv


def load_ipos_and_scores():
    client = MongoClient(MONGO_URI)
    ipos = list(client[DB_NAME][COL_IPOS].find({}))
    analysis = list(client[DB_NAME][COL_ANALYSIS].find({"status": "scored"}))
    client.close()

    ipos_by_name = {d["ipo"]: d for d in ipos if "ipo" in d}
    scored_by_name = {a["ipo"]: a for a in analysis if "ipo" in a}
    return ipos_by_name, scored_by_name


def parse_issue_price(ipo_doc):
    """Extract mid price from issue_price dict or string."""
    v = ipo_doc.get("issue_price") or ipo_doc.get("extracted_fields", {}).get("Price Band")
    if isinstance(v, dict):
        nums = [v.get("avg") or v.get("mid") or v.get("min")]
    else:
        nums = re.findall(r"\d+\.?\d*", str(v))
    nums = [safe_float(n) for n in nums if safe_float(n) is not None]
    return sum(nums) / len(nums) if nums else None


# ---------- BUILD IPO CANDIDATES ----------
def prepare_candidates(ipos_by_name, scored_by_name, hold_until):
    candidates = []

    for ipo_name, ipo_doc in ipos_by_name.items():
        if ipo_name not in scored_by_name:
            continue

        analysis = scored_by_name[ipo_name]
        fields = ipo_doc.get("extracted_fields", {}) or {}
        score = analysis.get("score")

        # Skip incomplete records
        if not score:
            continue

        # Close date
        close_str = (
            ipo_doc.get("close_date")
            or fields.get("Close Date")
            or fields.get("IPO Dates")
            or ""
        )
        date_match = re.findall(r"\d{4}-\d{2}-\d{2}", close_str)
        close_date = try_parse_date(date_match[0]) if date_match else try_parse_date(close_str)
        if not close_date or close_date > hold_until:
            continue

        category = ipo_doc.get("category", "Mainboard").lower()
        issue_mid = parse_issue_price(ipo_doc)
        if not issue_mid:
            continue

        lot_text = fields.get("Market Lot & Amounts")
        lot, min_invest = parse_lot_and_min_invest(lot_text)

        # Determine final minimum investment
        if "sme" in category:
            if not min_invest and lot:
                min_invest = lot * issue_mid
        else:
            # For Mainboard, fallback to Rs 15k
            if not min_invest:
                min_invest = MIN_INVEST_MAINBOARD
            # If parsed, ensure at least 15k (retail minimum)
            min_invest = max(min_invest, MIN_INVEST_MAINBOARD)

        if not min_invest:
            continue

        candidates.append({
            "ipo": ipo_name,
            "category": "SME" if "sme" in category else "Mainboard",
            "score": score,
            "issue_mid": issue_mid,
            "lot": int(lot) if lot else None,
            "min_invest": float(min_invest),
            "close_date": close_date,
            "gmp_pct": analysis.get("gmp_pct"),
        })

    return candidates


# ---------- GREEDY OPTIMIZATION ----------
def allocate_budget(candidates, budget):
    for c in candidates:
        c["score_per_inr"] = c["score"] / c["min_invest"]

    candidates = sorted(candidates, key=lambda x: x["score_per_inr"], reverse=True)
    allocation = []
    remaining = budget

    for c in candidates:
        if remaining >= c["min_invest"]:
            lots = int(remaining // c["min_invest"])
            invested = lots * c["min_invest"]
            remaining -= invested

            allocation.append({
                "ipo": c["ipo"],
                "score": c["score"],
                "issue_mid": c["issue_mid"],
                "lots": lots,
                "min_invest": c["min_invest"],
                "invested": invested,
                "category": c["category"],
            })

    return allocation, remaining


# ---------- MAIN ----------
def main():
    print("📈 IPO Portfolio Optimizer for Retail Investors\n")

    try:
        budget = float(input("💰 Enter your total investment budget (e.g., 100000): ").strip())
        hold_date = datetime.strptime(input("📅 Enter your max hold date (YYYY-MM-DD): ").strip(), "%Y-%m-%d").date()
    except Exception:
        print("❌ Invalid input.")
        return

    ipos_by_name, scored_by_name = load_ipos_and_scores()
    candidates = prepare_candidates(ipos_by_name, scored_by_name, hold_date)

    if not candidates:
        print("⚠️ No valid IPOs found for your criteria.")
        return

    print(f"\n✅ Found {len(candidates)} eligible IPOs:\n")
    for c in candidates:
        print(f"• {c['ipo']} | {c['category']} | Score: {c['score']} | Min Invest: ₹{int(c['min_invest'])} | Close: {c['close_date']}")

    allocation, leftover = allocate_budget(candidates, budget)

    print("\n📊 Final Allocation Plan:")
    total_invested = 0
    for a in allocation:
        total_invested += a["invested"]
        print(f"  - {a['ipo']} ({a['category']}): {a['lots']} lot(s), ₹{int(a['invested'])} invested, score {a['score']}")

    print(f"\n💵 Total Invested: ₹{int(total_invested)}")
    print(f"💤 Remaining Unused: ₹{int(leftover)}")

    # Save to MongoDB
    client = MongoClient(MONGO_URI)
    rec = {
        "created_at": datetime.utcnow().isoformat(),
        "budget": budget,
        "hold_until": hold_date.isoformat(),
        "allocation": allocation,
        "total_invested": total_invested,
        "leftover": leftover,
    }
    client[DB_NAME][COL_RECOMMEND].insert_one(rec)
    client.close()

    print("\n📦 Recommendation saved to MongoDB (collection: ipo_portfolio_recommendations)")
    print("✅ Done.")


if __name__ == "__main__":
    main()


📈 IPO Portfolio Optimizer for Retail Investors


✅ Found 6 eligible IPOs:

• PhysicsWallah | Mainboard | Score: 5.8 | Min Invest: ₹15000 | Close: 2025-11-13
• Emmvee Photovoltaic Power | Mainboard | Score: 5.1 | Min Invest: ₹15000 | Close: 2025-11-13
• Workmates Core2Cloud | SME | Score: 4.9 | Min Invest: ₹244800 | Close: 2025-11-13
• Tenneco Clean Air | Mainboard | Score: 4.7 | Min Invest: ₹15000 | Close: 2025-11-14
• Shining Tools | SME | Score: 4.9 | Min Invest: ₹273600 | Close: 2025-11-11
• Shreeji Global FMCG | SME | Score: 4.7 | Min Invest: ₹250000 | Close: 2025-11-07

📊 Final Allocation Plan:
  - PhysicsWallah (Mainboard): 6 lot(s), ₹90000 invested, score 5.8

💵 Total Invested: ₹90000
💤 Remaining Unused: ₹10000

📦 Recommendation saved to MongoDB (collection: ipo_portfolio_recommendations)
✅ Done.


In [3]:
import os
import re
from datetime import datetime
from pymongo import MongoClient

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_ANALYSIS = "ipo_analysis"
COL_RECOMMEND = "ipo_portfolio_recommendations"

MIN_INVEST_MAINBOARD = 15000


# ---------- HELPERS ----------
def safe_float(x):
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except Exception:
        return None


def try_parse_date(s):
    if not s:
        return None
    s = str(s).strip()
    fmts = ["%Y-%m-%d", "%d-%b-%Y", "%d %b %Y", "%d/%m/%Y"]
    for f in fmts:
        try:
            return datetime.strptime(s[:len(f)], f).date()
        except Exception:
            continue
    try:
        import dateutil.parser
        return dateutil.parser.parse(s, dayfirst=True).date()
    except Exception:
        return None


def parse_lot_and_min_invest(text):
    """Parse lot size and min invest from strings like 'Min: 137 shares/₹14,933'"""
    if not text:
        return None, None
    text = str(text)
    lot = None
    min_inv = None
    m = re.search(r"(\d{1,5})\s*shares?.*?₹\s?([\d,]+)", text)
    if m:
        lot = safe_float(m.group(1))
        min_inv = safe_float(m.group(2))
    else:
        m2 = re.search(r"₹\s?([\d,]+)", text)
        if m2:
            min_inv = safe_float(m2.group(1))
    return lot, min_inv


def load_data():
    client = MongoClient(MONGO_URI)
    ipos = list(client[DB_NAME][COL_IPOS].find({}))
    analysis = list(client[DB_NAME][COL_ANALYSIS].find({"status": "scored"}))
    client.close()
    ipos_by_name = {d["ipo"]: d for d in ipos if "ipo" in d}
    scored = {a["ipo"]: a for a in analysis if "ipo" in a}
    return ipos_by_name, scored


# ---------- FACTOR EXTRACTORS ----------
def extract_retail_quota(text):
    """Extract Retail % from 'Investor Quota Split: QIB 75%, NII 15%, Retail 10%'"""
    if not text:
        return 10
    m = re.search(r"Retail\s*:?(\d+\.?\d*)%", text, flags=re.I)
    if m:
        return safe_float(m.group(1))
    return 10.0


def extract_fundamental_score(text):
    """
    Crude rule-based scoring based on Financial Performance & Valuation Ratios.
    Higher EPS, ROE, lower D/E = better.
    """
    if not text:
        return 5
    score = 5.0
    t = str(text).lower()

    if "profit" in t or "positive" in t or "growth" in t:
        score += 2
    if "loss" in t or "negative" in t:
        score -= 2
    if "roe" in t:
        m = re.search(r"roe[:\s]*([-\d.]+)", t)
        if m:
            roe = safe_float(m.group(1))
            if roe and roe > 10:
                score += 1.5
    if "d/e" in t:
        m = re.search(r"d/e[:\s]*([-\d.]+)", t)
        if m:
            de = safe_float(m.group(1))
            if de and de > 1:
                score -= 1
    if "eps" in t:
        m = re.search(r"eps[:\s]*[-(₹]?([\d.]+)", t)
        if m:
            eps = safe_float(m.group(1))
            if eps and eps > 0:
                score += 1
            else:
                score -= 1
    return max(1, min(score, 10))


def extract_sentiment_from_overview(text):
    """Very crude semantic scoring based on company description."""
    if not text:
        return 5
    t = str(text).lower()
    good = ["growing", "leader", "expanding", "innovative", "strong", "profitable", "stable"]
    bad = ["loss", "decline", "volatile", "uncertain", "risky", "unprofitable"]
    score = 5
    for w in good:
        if w in t:
            score += 0.5
    for w in bad:
        if w in t:
            score -= 0.5
    return max(1, min(score, 10))


def compute_composite_score(ipo_doc, analysis):
    fields = ipo_doc.get("extracted_fields", {}) or {}
    base_score = analysis.get("score", 5)

    retail_quota = extract_retail_quota(fields.get("Investor Quota Split"))
    fund_score = extract_fundamental_score(
        fields.get("Valuation Ratios (EPS, ROE, ROCE, D/E, NAV)")
        or fields.get("Financial Performance (FY23–FY25)")
    )
    sentiment_score = extract_sentiment_from_overview(fields.get("Company Overview"))
    gmp = safe_float(ipo_doc.get("gmp_investorgain")) or 0
    issue = ipo_doc.get("issue_price", {}).get("avg") if isinstance(ipo_doc.get("issue_price"), dict) else None
    gmp_strength = (gmp / issue * 100) if (gmp and issue) else 0

    # Normalize retail quota to 0–10 scale
    rq_score = min(retail_quota / 10, 1) * 10

    # Weighted composite
    composite = (
        0.30 * base_score +
        0.25 * rq_score +
        0.20 * fund_score +
        0.15 * (gmp_strength / 10) +
        0.10 * sentiment_score
    )
    return round(min(composite, 10), 2)


# ---------- PREPARE CANDIDATES ----------
def prepare_candidates(ipos_by_name, scored_by_name, hold_until):
    candidates = []

    for name, ipo_doc in ipos_by_name.items():
        if name not in scored_by_name:
            continue
        analysis = scored_by_name[name]
        fields = ipo_doc.get("extracted_fields", {}) or {}

        close_str = ipo_doc.get("close_date") or fields.get("Close Date") or fields.get("IPO Dates")
        close_date = try_parse_date(close_str)
        if not close_date or close_date > hold_until:
            continue

        category = ipo_doc.get("category", "Mainboard").lower()
        lot, min_invest = parse_lot_and_min_invest(fields.get("Market Lot & Amounts"))
        issue_mid = ipo_doc.get("issue_price", {}).get("avg") if isinstance(ipo_doc.get("issue_price"), dict) else None
        if not issue_mid:
            continue

        if not min_invest:
            min_invest = MIN_INVEST_MAINBOARD

        composite_score = compute_composite_score(ipo_doc, analysis)

        candidates.append({
            "ipo": name,
            "category": "SME" if "sme" in category else "Mainboard",
            "score": composite_score,
            "issue_mid": issue_mid,
            "lot": lot,
            "min_invest": float(min_invest),
            "close_date": close_date,
            "retail_quota": extract_retail_quota(fields.get("Investor Quota Split")),
        })

    return candidates


# ---------- ALLOCATION (NEW MAX UTILIZATION VERSION) ----------
def allocate_budget(candidates, budget):
    for c in candidates:
        c["score_per_inr"] = c["score"] / c["min_invest"]
    candidates = sorted(candidates, key=lambda x: x["score_per_inr"], reverse=True)

    allocation = []
    remaining = budget

    # Pass 1 — Diversify: give one minimum lot to each IPO if possible
    for c in candidates:
        if remaining >= c["min_invest"]:
            allocation.append({
                "ipo": c["ipo"],
                "category": c["category"],
                "score": c["score"],
                "lots": 1,
                "invested": c["min_invest"],
                "min_invest": c["min_invest"],
                "retail_quota": c["retail_quota"]
            })
            remaining -= c["min_invest"]

    # Pass 2 — Maximize: add more lots where score_per_inr is highest
    while True:
        affordable = [c for c in candidates if c["min_invest"] <= remaining]
        if not affordable:
            break

        best = max(affordable, key=lambda x: x["score_per_inr"])
        for alloc in allocation:
            if alloc["ipo"] == best["ipo"]:
                alloc["lots"] += 1
                alloc["invested"] += best["min_invest"]
                remaining -= best["min_invest"]
                break
        else:
            allocation.append({
                "ipo": best["ipo"],
                "category": best["category"],
                "score": best["score"],
                "lots": 1,
                "invested": best["min_invest"],
                "min_invest": best["min_invest"],
                "retail_quota": best["retail_quota"]
            })
            remaining -= best["min_invest"]

        if remaining < min(c["min_invest"] for c in candidates):
            break

    # Smart rebalance if large leftover (>10% budget)
    if remaining > 0.1 * budget and allocation:
        lowest = min(allocation, key=lambda x: x["score"], default=None)
        if lowest:
            removed_amt = lowest["invested"]
            allocation.remove(lowest)
            remaining += removed_amt
            affordable = [c for c in candidates if c["min_invest"] <= remaining]
            for best in sorted(affordable, key=lambda x: x["score_per_inr"], reverse=True):
                if remaining < best["min_invest"]:
                    continue
                allocation.append({
                    "ipo": best["ipo"],
                    "category": best["category"],
                    "score": best["score"],
                    "lots": 1,
                    "invested": best["min_invest"],
                    "min_invest": best["min_invest"],
                    "retail_quota": best["retail_quota"]
                })
                remaining -= best["min_invest"]
                if remaining < min(c["min_invest"] for c in candidates):
                    break

    return allocation, remaining


# ---------- MAIN ----------
def main():
    print("📊 IPO Portfolio Optimizer (Enhanced: Fundamentals, Retail Probability & Max Utilization)\n")

    try:
        budget = float(input("💰 Enter your total investment budget (e.g., 100000): ").strip())
        hold_until = datetime.strptime(input("📅 Enter your max hold date (YYYY-MM-DD): ").strip(), "%Y-%m-%d").date()
    except Exception:
        print("❌ Invalid input.")
        return

    ipos_by_name, scored_by_name = load_data()
    candidates = prepare_candidates(ipos_by_name, scored_by_name, hold_until)
    if not candidates:
        print("⚠️ No valid IPOs found.")
        return

    print(f"✅ Found {len(candidates)} eligible IPOs:\n")
    for c in candidates:
        print(f"• {c['ipo']} | {c['category']} | Score: {c['score']} | Min ₹{int(c['min_invest'])} | Retail {c['retail_quota']}%")

    allocation, leftover = allocate_budget(candidates, budget)

    print("\n📈 Optimized Allocation Plan:")
    total_invested = 0
    for a in allocation:
        total_invested += a["invested"]
        print(f"  - {a['ipo']}: ₹{int(a['invested'])} ({a['lots']} lots) | Score: {a['score']} | Retail {a['retail_quota']}%")

    print(f"\n💵 Total Invested: ₹{int(total_invested)}")
    print(f"💤 Remaining: ₹{int(leftover)}")

    # Save recommendation
    client = MongoClient(MONGO_URI)
    client[DB_NAME][COL_RECOMMEND].insert_one({
        "created_at": datetime.utcnow().isoformat(),
        "budget": budget,
        "hold_until": hold_until.isoformat(),
        "allocation": allocation,
        "total_invested": total_invested,
        "leftover": leftover
    })
    client.close()

    print("\n📦 Recommendation saved to MongoDB.")
    print("✅ Done.")


if __name__ == "__main__":
    main()


📊 IPO Portfolio Optimizer (Enhanced: Fundamentals, Retail Probability & Max Utilization)

✅ Found 6 eligible IPOs:

• PhysicsWallah | Mainboard | Score: 6.3 | Min ₹14933 | Retail 10.0%
• Emmvee Photovoltaic Power | Mainboard | Score: 6.17 | Min ₹14973 | Retail 10.0%
• Workmates Core2Cloud | SME | Score: 6.13 | Min ₹244800 | Retail 10.0%
• Tenneco Clean Air | Mainboard | Score: 6.09 | Min ₹14689 | Retail 10.0%
• Shining Tools | SME | Score: 6.03 | Min ₹273600 | Retail 10.0%
• Shreeji Global FMCG | SME | Score: 5.91 | Min ₹250000 | Retail 10.0%

📈 Optimized Allocation Plan:
  - PhysicsWallah: ₹194129 (13 lots) | Score: 6.3 | Retail 10.0%
  - Tenneco Clean Air: ₹14689 (1 lots) | Score: 6.09 | Retail 10.0%
  - Emmvee Photovoltaic Power: ₹14973 (1 lots) | Score: 6.17 | Retail 10.0%
  - Workmates Core2Cloud: ₹244800 (1 lots) | Score: 6.13 | Retail 10.0%
  - Shreeji Global FMCG: ₹250000 (1 lots) | Score: 5.91 | Retail 10.0%
  - Shining Tools: ₹273600 (1 lots) | Score: 6.03 | Retail 10.0%

💵 T

In [5]:
"""
IPO allocator (final):

- Filters: score >= 5; close_date <= user hold date.
- Attempts to use all budget (unless no eligible IPOs).
- Primary solver: pulp MILP (integer lots).
- Fallback solver: greedy + repair to fully utilize budget.
- Explainability: reasons for more/less allocation per IPO.
- Persists recommendation to MongoDB.

Requires: pymongo. Optional: pulp (recommended).
"""

import os
import re
from datetime import datetime
from pymongo import MongoClient

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_ANALYSIS = "ipo_analysis"
COL_RECOMMEND = "ipo_portfolio_recommendations"

MIN_INVEST_MAINBOARD = 15000
# max lots per IPO to avoid over-concentration (retail behaviour) — tweakable
DEFAULT_MAX_LOTS_PER_IPO = 3

# ---------- HELPERS ----------
def safe_float(x):
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except Exception:
        return None

def try_parse_date(s):
    if not s:
        return None
    s = str(s).strip()
    fmts = ["%Y-%m-%d", "%d-%b-%Y", "%d %b %Y", "%d/%m/%Y"]
    for f in fmts:
        try:
            return datetime.strptime(s[:len(f)], f).date()
        except Exception:
            pass
    try:
        import dateutil.parser
        return dateutil.parser.parse(s, dayfirst=True).date()
    except Exception:
        return None

def parse_lot_and_min_invest(text):
    """Return (lot, min_invest) from Market Lot & Amounts string."""
    if not text:
        return None, None
    s = str(text)
    # look for 'Min: 137 shares/₹14,933' or '137 shares/₹14,933'
    m = re.search(r"(?:Min[:\s]*)?(\d{1,6})\s*shares?.*?₹\s?([\d,]+)", s, flags=re.I)
    if m:
        lot = safe_float(m.group(1))
        min_inv = safe_float(m.group(2))
        return int(lot) if lot else None, float(min_inv) if min_inv else None
    # fallback: find first '₹' number
    m2 = re.search(r"₹\s?([\d,]+)", s)
    if m2:
        return None, float(safe_float(m2.group(1)))
    return None, None

def parse_issue_mid(ipo_doc):
    v = ipo_doc.get("issue_price") or ipo_doc.get("extracted_fields", {}).get("Price Band")
    if isinstance(v, dict):
        mid = v.get("avg") or v.get("mid") or v.get("min")
        return safe_float(mid)
    if not v:
        return None
    nums = re.findall(r"\d+\.?\d*", str(v))
    nums = [safe_float(n) for n in nums if safe_float(n) is not None]
    return sum(nums)/len(nums) if nums else None

def load_data():
    client = MongoClient(MONGO_URI)
    ipos = list(client[DB_NAME][COL_IPOS].find({}))
    analysis = list(client[DB_NAME][COL_ANALYSIS].find({}))
    client.close()
    ipos_by_name = {d["ipo"]: d for d in ipos if "ipo" in d}
    scored = {a["ipo"]: a for a in analysis if "ipo" in a}
    return ipos_by_name, scored

# composite score extractor reused from previous pipeline:
def extract_retail_quota(text):
    if not text:
        return 10.0
    m = re.search(r"Retail\s*:?(\d+\.?\d*)%", str(text), flags=re.I)
    if m:
        return safe_float(m.group(1))
    return 10.0

def extract_fundamental_score(text):
    if not text:
        return 5.0
    score = 5.0
    t = str(text).lower()
    if "profit" in t or "positive" in t or "growth" in t:
        score += 2
    if "loss" in t or "negative" in t:
        score -= 2
    if "roe" in t:
        m = re.search(r"roe[:\s]*([-\d.]+)", t)
        if m:
            roe = safe_float(m.group(1))
            if roe and roe > 10:
                score += 1.5
    if "d/e" in t:
        m = re.search(r"d/e[:\s]*([-\d.]+)", t)
        if m:
            de = safe_float(m.group(1))
            if de and de > 1:
                score -= 1
    if "eps" in t:
        m = re.search(r"eps[:\s]*[-(₹]?([\d.]+)", t)
        if m:
            eps = safe_float(m.group(1))
            if eps and eps > 0:
                score += 1
            else:
                score -= 1
    return max(1, min(score, 10))

def extract_sentiment(text):
    if not text:
        return 5.0
    t = str(text).lower()
    score = 5.0
    good = ["growing", "leader", "expanding", "innovative", "strong", "profitable", "stable"]
    bad = ["loss", "decline", "volatile", "uncertain", "risky", "unprofitable"]
    for w in good:
        if w in t:
            score += 0.5
    for w in bad:
        if w in t:
            score -= 0.5
    return max(1, min(score, 10))

def compute_composite(ipo_doc, analysis):
    fields = ipo_doc.get("extracted_fields", {}) or {}
    base = analysis.get("score", 5)
    retail = extract_retail_quota(fields.get("Investor Quota Split"))
    fund = extract_fundamental_score(fields.get("Valuation Ratios (EPS, ROE, ROCE, D/E, NAV)") or fields.get("Financial Performance (FY23–FY25)"))
    sent = extract_sentiment(fields.get("Company Overview"))
    gmp = safe_float(ipo_doc.get("gmp_investorgain")) or 0
    issue = parse_issue_mid(ipo_doc)
    gmp_strength = (gmp / issue * 100) if (gmp and issue) else 0
    rq_score = min(retail / 10, 1) * 10
    composite = 0.30*base + 0.25*rq_score + 0.20*fund + 0.15*(gmp_strength/10) + 0.10*sent
    return round(min(composite, 10), 3), retail, fund, sent, gmp_strength

# ---------- ALLOCATION CORE ----------
def build_candidates(ipos_by_name, scored, hold_date):
    cands = []
    for name, ipo in ipos_by_name.items():
        if name not in scored:
            continue
        analysis = scored[name]
        if analysis.get("status") != "scored" and "score" not in analysis:
            # skip not scored
            continue
        fields = ipo.get("extracted_fields", {}) or {}
        # close date parse
        close = ipo.get("close_date") or fields.get("IPO Dates") or fields.get("Close Date") or ipo.get("extracted_fields", {}).get("IPO Dates")
        close_date = None
        if close:
            # try to extract year-month-day if present
            m = re.search(r"\d{4}-\d{2}-\d{2}", str(close))
            if m:
                close_date = try_parse_date(m.group(0))
            else:
                close_date = try_parse_date(close)
        if not close_date:
            # can't use if no reliable close date
            continue
        if close_date > hold_date:
            # user can't hold that long
            continue

        issue_mid = parse_issue_mid(ipo)
        if not issue_mid:
            continue

        lot, min_inv = parse_lot_and_min_invest(fields.get("Market Lot & Amounts"))
        if not min_inv:
            # fallback for mainboard: lot*issue_mid preferred, otherwise MIN_INVEST_MAINBOARD
            if lot:
                min_inv = lot * issue_mid
            else:
                min_inv = MIN_INVEST_MAINBOARD

        composite_score, retail_quota, fund_score, sent_score, gmp_strength = compute_composite(ipo, analysis)
        # filter by composite >= 5
        if composite_score < 5:
            continue

        # caps for lots (avoid insane concentration)
        max_lots = int(max(1, min(DEFAULT_MAX_LOTS_PER_IPO, (1000000000 if min_inv==0 else 10**9))))  # will be tightened later by budget
        # final candidate
        cands.append({
            "ipo": name,
            "category": ipo.get("category", "Mainboard"),
            "composite": composite_score,
            "retail_quota": retail_quota,
            "fund_score": fund_score,
            "sentiment": sent_score,
            "gmp_strength": gmp_strength,
            "issue_mid": issue_mid,
            "lot": int(lot) if lot else None,
            "min_invest": float(min_inv),
            "close_date": close_date,
            "analysis": analysis
        })
    return cands

def solve_with_pulp(candidates, budget):
    try:
        import pulp
    except Exception:
        return None  # fallback if pulp not installed

    prob = pulp.LpProblem("IPO_Allocation", pulp.LpMaximize)

    lot_vars = {}
    for c in candidates:
        max_possible = int(max(1, budget // c["min_invest"]))
        cap = min(DEFAULT_MAX_LOTS_PER_IPO, max_possible)

        # fix: build safe variable name outside f-string
        safe_name = re.sub(r"\W+", "_", c["ipo"])
        var = pulp.LpVariable(f"lots_{safe_name}", lowBound=0, upBound=cap, cat="Integer")
        lot_vars[c["ipo"]] = var

    # objective: maximize weighted total score
    prob += pulp.lpSum([c["composite"] * lot_vars[c["ipo"]] for c in candidates])

    # budget constraint
    prob += pulp.lpSum([c["min_invest"] * lot_vars[c["ipo"]] for c in candidates]) <= budget

    # solve using CBC solver
    prob.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=10))

    # collect solution
    allocation = []
    total_invested = 0.0
    for c in candidates:
        v = int(pulp.value(lot_vars[c["ipo"]]) or 0)
        if v > 0:
            invested = v * c["min_invest"]
            allocation.append({
                "ipo": c["ipo"],
                "lots": v,
                "invested": invested,
                "min_invest": c["min_invest"],
                "composite": c["composite"],
                "retail_quota": c["retail_quota"],
                "reason_fields": c
            })
            total_invested += invested

    remaining = budget - total_invested
    return allocation, remaining


# fallback greedy + repair that fully uses budget
def greedy_fill_full(candidates, budget):
    # sort by composite per rupee descending; start by giving 1 lot to top N until can't
    cand_sorted = sorted(candidates, key=lambda x: x["composite"]/x["min_invest"], reverse=True)
    allocation = []
    remaining = budget

    # initial pass: try one lot each to top candidates while possible
    for c in cand_sorted:
        if remaining >= c["min_invest"]:
            allocation.append({"ipo": c["ipo"], "lots": 1, "min_invest": c["min_invest"], "invested": c["min_invest"], "composite": c["composite"], "retail_quota": c["retail_quota"], "reason_fields": c})
            remaining -= c["min_invest"]

    # second pass: attempt to add lots to highest marginal benefit
    while True:
        affordable = [c for c in cand_sorted if c["min_invest"] <= remaining]
        if not affordable:
            break
        best = max(affordable, key=lambda x: x["composite"]/x["min_invest"])
        # find alloc entry
        found = next((a for a in allocation if a["ipo"] == best["ipo"]), None)
        if found:
            found["lots"] += 1
            found["invested"] += best["min_invest"]
        else:
            allocation.append({"ipo": best["ipo"], "lots": 1, "min_invest": best["min_invest"], "invested": best["min_invest"], "composite": best["composite"], "retail_quota": best["retail_quota"], "reason_fields": best})
        remaining -= best["min_invest"]

    # repair: if remaining > 0 but < min(min_invest), attempt to rebalance:
    min_unit = min([c["min_invest"] for c in candidates]) if candidates else 0
    if remaining > 0 and remaining < min_unit:
        # try swapping out the lowest composite-per-rupee allocated lot
        # pick allocated entry with smallest marginal benefit
        allocation_sorted = sorted(allocation, key=lambda a: a["composite"]/a["min_invest"])
        for out in allocation_sorted:
            if out["lots"] <= 0:
                continue
            # remove one lot
            out["lots"] -= 1
            out["invested"] -= out["min_invest"]
            remaining += out["min_invest"]
            if out["lots"] == 0:
                allocation = [x for x in allocation if x["ipo"] != out["ipo"]]
            # now try to fill with candidate(s)
            filled = False
            for c in cand_sorted:
                while remaining >= c["min_invest"]:
                    # add lot
                    found = next((a for a in allocation if a["ipo"] == c["ipo"]), None)
                    if found:
                        found["lots"] += 1
                        found["invested"] += c["min_invest"]
                    else:
                        allocation.append({"ipo": c["ipo"], "lots": 1, "min_invest": c["min_invest"], "invested": c["min_invest"], "composite": c["composite"], "retail_quota": c["retail_quota"], "reason_fields": c})
                    remaining -= c["min_invest"]
                    filled = True
            if filled:
                break
        # if still leftover and cannot allocate, leave small remainder
    return allocation, remaining

# generate explainability text
def explain_allocation(allocation, candidates_dict):
    explain = {}
    for a in allocation:
        c = candidates_dict.get(a["ipo"])
        reasons_more = []
        reasons_less = []
        # why more: high composite, decent retail quota, good fundamentals, GMP strength
        if c:
            if c["composite"] >= 7:
                reasons_more.append("High composite score (strong fundamentals + allotment chance + GMP).")
            elif c["composite"] >= 6:
                reasons_more.append("Above-average composite score.")
            else:
                reasons_more.append("Moderate composite score.")
            if c["retail_quota"] >= 30:
                reasons_more.append(f"Retail quota is {c['retail_quota']}% — good allotment chance.")
            else:
                reasons_less.append(f"Retail quota is low ({c['retail_quota']}%) — allotment chance may be lower.")
            if c["fund_score"] and c["fund_score"] >= 6:
                reasons_more.append("Fundamentals look decent.")
            if c["gmp_strength"] and c["gmp_strength"] > 10:
                reasons_more.append(f"GMP indicates listing strength (~{round(c['gmp_strength'],1)}%).")
            # SME note
            if "sme" in str(c.get("category","")).lower():
                reasons_less.append("SME IPO: higher risk, less liquidity — keep small exposure.")
        explain[a["ipo"]] = {"reasons_more": reasons_more, "reasons_less": reasons_less}
    return explain

# ---------- MAIN ----------
def main():
    print("IPO Allocator — final (no leftovers unless all poor)\n")
    try:
        budget = float(input("Enter total budget in INR (e.g., 120000): ").strip())
        hold_until = datetime.strptime(input("Enter hold-until date (YYYY-MM-DD): ").strip(), "%Y-%m-%d").date()
    except Exception as e:
        print("Invalid input:", e)
        return

    ipos_by_name, scored = load_data()
    candidates = build_candidates(ipos_by_name, scored, hold_until)

    if not candidates:
        print("No eligible IPOs (or all below score threshold / missing dates). Exiting.")
        return

    # Tighten max_lots per IPO based on budget
    for c in candidates:
        max_possible = int(budget // c["min_invest"]) if c["min_invest"]>0 else 0
        c["max_lots"] = max(0, min(DEFAULT_MAX_LOTS_PER_IPO, max_possible))

    # remove candidates with max_lots==0 (too expensive)
    candidates = [c for c in candidates if c["max_lots"]>0]
    if not candidates:
        print("All eligible IPOs have min investment > budget. Exiting.")
        return

    print(f"Eligible IPOs (score >=5 and within hold date):")
    for c in candidates:
        print(f" • {c['ipo']}: composite={c['composite']}, min_invest=₹{int(c['min_invest'])}, retail={c['retail_quota']}% , max_lots={c['max_lots']}")

    # try pulp MILP
    pulp_result = solve_with_pulp(candidates, budget)
    if pulp_result is not None:
        allocation, remaining = pulp_result
        # if solution left a lot of leftover, try greedy_fill_full as alternative and pick better utilization
        if remaining > 0.05 * budget:
            g_alloc, g_rem = greedy_fill_full(candidates, budget)
            # choose whichever uses more budget (smaller remainder), tie-break on higher objective
            def used(a): return sum(x["invested"] for x in a)
            if (budget - g_rem) > (budget - remaining):
                allocation, remaining = g_alloc, g_rem
    else:
        allocation, remaining = greedy_fill_full(candidates, budget)

    # ensure we invested something — if nothing, stop
    if not allocation:
        print("Could not allocate to any IPO (budget too small vs min_invest).")
        return

    # Build explainability
    candidates_dict = {c["ipo"]: c for c in candidates}
    explain = explain_allocation(allocation, candidates_dict)

    total_invested = sum(a["invested"] for a in allocation)
    print("\n--- Allocation ---")
    for a in allocation:
        print(f"{a['ipo']}: lots={a['lots']}, invested=₹{int(a['invested'])}, min_unit=₹{int(a['min_invest'])}, score={round(a['composite'],2) if 'composite' in a else 'N/A'}")
    print(f"\nTotal invested: ₹{int(total_invested)}")
    print(f"Remaining (uninvested): ₹{int(remaining)}")

    # print explainability per IPO
    print("\n--- Explainability ---")
    for ipo, ex in explain.items():
        print(f"\n{ipo}:")
        if ex["reasons_more"]:
            print("  Reasons to invest more:")
            for r in ex["reasons_more"]:
                print("   -", r)
        if ex["reasons_less"]:
            print("  Reasons to be cautious / invest less:")
            for r in ex["reasons_less"]:
                print("   -", r)

    # save recommendation
    client = MongoClient(MONGO_URI)
    rec = {
        "created_at": datetime.utcnow().isoformat(),
        "budget": budget,
        "hold_until": hold_until.isoformat(),
        "allocation": allocation,
        "explain": explain,
        "total_invested": total_invested,
        "leftover": remaining
    }
    client[DB_NAME][COL_RECOMMEND].insert_one(rec)
    client.close()
    print("\nSaved recommendation to MongoDB (collection:", COL_RECOMMEND, ")")
    print("Done.")

if __name__ == "__main__":
    main()


IPO Allocator — final (no leftovers unless all poor)

Eligible IPOs (score >=5 and within hold date):
 • PhysicsWallah: composite=6.3, min_invest=₹14933, retail=10.0% , max_lots=3
 • Emmvee Photovoltaic Power: composite=6.171, min_invest=₹14973, retail=10.0% , max_lots=3
 • Workmates Core2Cloud: composite=6.135, min_invest=₹244800, retail=10.0% , max_lots=3
 • Tenneco Clean Air: composite=6.089, min_invest=₹14689, retail=10.0% , max_lots=3
 • Shining Tools: composite=6.028, min_invest=₹273600, retail=10.0% , max_lots=3
 • Shreeji Global FMCG: composite=5.91, min_invest=₹250000, retail=10.0% , max_lots=3

--- Allocation ---
PhysicsWallah: lots=31, invested=₹462923, min_unit=₹14933, score=6.3
Tenneco Clean Air: lots=1, invested=₹14689, min_unit=₹14689, score=6.09
Emmvee Photovoltaic Power: lots=1, invested=₹14973, min_unit=₹14973, score=6.17
Workmates Core2Cloud: lots=1, invested=₹244800, min_unit=₹244800, score=6.13
Shreeji Global FMCG: lots=1, invested=₹250000, min_unit=₹250000, score=

InvalidDocument: cannot encode object: datetime.date(2025, 11, 13), of type: <class 'datetime.date'>

In [6]:
import os
import re
from datetime import datetime, date
from pymongo import MongoClient

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_ANALYSIS = "ipo_analysis"
COL_RECOMMEND = "ipo_portfolio_recommendations"

MIN_INVEST_MAINBOARD = 15000
DEFAULT_MAX_LOTS_PER_IPO = 3   # retail-friendly cap
DIVERSIFICATION_WEIGHT = 0.1   # penalty for over-concentration

# ---------- HELPERS ----------
def safe_float(x):
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except Exception:
        return None

def try_parse_date(s):
    if not s:
        return None
    s = str(s).strip()
    fmts = ["%Y-%m-%d", "%d-%b-%Y", "%d %b %Y", "%d/%m/%Y"]
    for f in fmts:
        try:
            return datetime.strptime(s[:len(f)], f).date()
        except Exception:
            pass
    try:
        import dateutil.parser
        return dateutil.parser.parse(s, dayfirst=True).date()
    except Exception:
        return None

def parse_lot_and_min_invest(text):
    if not text:
        return None, None
    s = str(text)
    m = re.search(r"(?:Min[:\s]*)?(\d{1,6})\s*shares?.*?₹\s?([\d,]+)", s, flags=re.I)
    if m:
        lot = safe_float(m.group(1))
        min_inv = safe_float(m.group(2))
        return int(lot) if lot else None, float(min_inv) if min_inv else None
    m2 = re.search(r"₹\s?([\d,]+)", s)
    if m2:
        return None, float(safe_float(m2.group(1)))
    return None, None

def parse_issue_mid(ipo_doc):
    v = ipo_doc.get("issue_price") or ipo_doc.get("extracted_fields", {}).get("Price Band")
    if isinstance(v, dict):
        mid = v.get("avg") or v.get("mid") or v.get("min")
        return safe_float(mid)
    if not v:
        return None
    nums = re.findall(r"\d+\.?\d*", str(v))
    nums = [safe_float(n) for n in nums if safe_float(n) is not None]
    return sum(nums)/len(nums) if nums else None

def sanitize_for_mongo(obj):
    """Recursively convert datetime.date to ISO string."""
    if isinstance(obj, dict):
        return {k: sanitize_for_mongo(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [sanitize_for_mongo(x) for x in obj]
    elif isinstance(obj, date):
        return datetime(obj.year, obj.month, obj.day).isoformat()
    else:
        return obj

def extract_retail_quota(text):
    if not text:
        return 10.0
    m = re.search(r"Retail\s*:?(\d+\.?\d*)%", str(text), flags=re.I)
    if m:
        return safe_float(m.group(1))
    return 10.0

def extract_fundamental_score(text):
    if not text:
        return 5.0
    score = 5.0
    t = str(text).lower()
    if "profit" in t or "positive" in t or "growth" in t:
        score += 2
    if "loss" in t or "negative" in t:
        score -= 2
    if "roe" in t:
        m = re.search(r"roe[:\s]*([-\d.]+)", t)
        if m:
            roe = safe_float(m.group(1))
            if roe and roe > 10:
                score += 1.5
    if "d/e" in t:
        m = re.search(r"d/e[:\s]*([-\d.]+)", t)
        if m:
            de = safe_float(m.group(1))
            if de and de > 1:
                score -= 1
    if "eps" in t:
        m = re.search(r"eps[:\s]*[-(₹]?([\d.]+)", t)
        if m:
            eps = safe_float(m.group(1))
            if eps and eps > 0:
                score += 1
            else:
                score -= 1
    return max(1, min(score, 10))

def extract_sentiment(text):
    if not text:
        return 5.0
    t = str(text).lower()
    score = 5.0
    good = ["growing", "leader", "expanding", "innovative", "strong", "profitable", "stable"]
    bad = ["loss", "decline", "volatile", "uncertain", "risky", "unprofitable"]
    for w in good:
        if w in t:
            score += 0.5
    for w in bad:
        if w in t:
            score -= 0.5
    return max(1, min(score, 10))

def compute_composite(ipo_doc, analysis):
    fields = ipo_doc.get("extracted_fields", {}) or {}
    base = analysis.get("score", 5)
    retail = extract_retail_quota(fields.get("Investor Quota Split"))
    fund = extract_fundamental_score(fields.get("Valuation Ratios (EPS, ROE, ROCE, D/E, NAV)") or fields.get("Financial Performance (FY23–FY25)"))
    sent = extract_sentiment(fields.get("Company Overview"))
    gmp = safe_float(ipo_doc.get("gmp_investorgain")) or 0
    issue = parse_issue_mid(ipo_doc)
    gmp_strength = (gmp / issue * 100) if (gmp and issue) else 0
    rq_score = min(retail / 10, 1) * 10
    composite = 0.30*base + 0.25*rq_score + 0.20*fund + 0.15*(gmp_strength/10) + 0.10*sent
    return round(min(composite, 10), 3), retail, fund, sent, gmp_strength

def load_data():
    client = MongoClient(MONGO_URI)
    ipos = list(client[DB_NAME][COL_IPOS].find({}))
    analysis = list(client[DB_NAME][COL_ANALYSIS].find({}))
    client.close()
    ipos_by_name = {d["ipo"]: d for d in ipos if "ipo" in d}
    scored = {a["ipo"]: a for a in analysis if "ipo" in a}
    return ipos_by_name, scored

# ---------- CANDIDATE PREP ----------
def build_candidates(ipos_by_name, scored, hold_date):
    cands = []
    for name, ipo in ipos_by_name.items():
        if name not in scored:
            continue
        analysis = scored[name]
        if analysis.get("status") != "scored" and "score" not in analysis:
            continue
        fields = ipo.get("extracted_fields", {}) or {}
        close = ipo.get("close_date") or fields.get("IPO Dates") or fields.get("Close Date")
        close_date = try_parse_date(str(close))
        if not close_date or close_date > hold_date:
            continue

        issue_mid = parse_issue_mid(ipo)
        if not issue_mid:
            continue

        lot, min_inv = parse_lot_and_min_invest(fields.get("Market Lot & Amounts"))
        if not min_inv:
            min_inv = lot * issue_mid if lot else MIN_INVEST_MAINBOARD

        composite, retail, fund, sent, gmp_strength = compute_composite(ipo, analysis)
        if composite < 5:
            continue

        cands.append({
            "ipo": name,
            "category": ipo.get("category", "Mainboard"),
            "composite": composite,
            "retail_quota": retail,
            "fund_score": fund,
            "sentiment": sent,
            "gmp_strength": gmp_strength,
            "issue_mid": issue_mid,
            "lot": lot,
            "min_invest": min_inv,
            "close_date": close_date,
        })
    return cands

# ---------- ALLOCATION ----------
def allocate_balanced(candidates, budget):
    """Balanced allocator that penalizes over-concentration and uses all budget."""
    try:
        import pulp
    except ImportError:
        print("⚠️ PuLP not installed, using fallback greedy mode.")
        return greedy_fill_full(candidates, budget)

    prob = pulp.LpProblem("Balanced_IPO_Allocation", pulp.LpMaximize)
    lot_vars = {}

    for c in candidates:
        safe_name = re.sub(r"\W+", "_", c["ipo"])
        max_possible = int(max(1, budget // c["min_invest"]))
        cap = min(DEFAULT_MAX_LOTS_PER_IPO, max_possible)
        lot_vars[c["ipo"]] = pulp.LpVariable(f"lots_{safe_name}", lowBound=0, upBound=cap, cat="Integer")

    # objective: maximize (score * lots) - diversification penalty
    total_lots = pulp.lpSum(lot_vars.values())
    objective = pulp.lpSum([c["composite"] * lot_vars[c["ipo"]] for c in candidates])
    penalty = DIVERSIFICATION_WEIGHT * pulp.lpSum([(lot_vars[c["ipo"]] ** 2) for c in candidates])
    prob += objective - penalty

    # budget constraint
    prob += pulp.lpSum([c["min_invest"] * lot_vars[c["ipo"]] for c in candidates]) <= budget

    prob.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=10))

    allocation = []
    total_invested = 0
    for c in candidates:
        v = int(pulp.value(lot_vars[c["ipo"]]) or 0)
        if v > 0:
            invested = v * c["min_invest"]
            allocation.append({
                "ipo": c["ipo"],
                "lots": v,
                "invested": invested,
                "min_invest": c["min_invest"],
                "composite": c["composite"],
                "retail_quota": c["retail_quota"]
            })
            total_invested += invested
    remaining = budget - total_invested
    return allocation, remaining

def greedy_fill_full(candidates, budget):
    candidates = sorted(candidates, key=lambda x: x["composite"]/x["min_invest"], reverse=True)
    allocation = []
    remaining = budget
    for c in candidates:
        if remaining < min(x["min_invest"] for x in candidates):
            break
        max_lots = min(DEFAULT_MAX_LOTS_PER_IPO, int(remaining // c["min_invest"]))
        if max_lots <= 0:
            continue
        invested = c["min_invest"] * max_lots
        allocation.append({
            "ipo": c["ipo"],
            "lots": max_lots,
            "invested": invested,
            "min_invest": c["min_invest"],
            "composite": c["composite"],
            "retail_quota": c["retail_quota"]
        })
        remaining -= invested
    return allocation, remaining

# ---------- MAIN ----------
def main():
    print("📊 Balanced IPO Allocator (Retail)\n")
    try:
        budget = float(input("Enter total budget (₹): ").strip())
        hold_date = datetime.strptime(input("Enter hold-until date (YYYY-MM-DD): ").strip(), "%Y-%m-%d").date()
    except Exception as e:
        print("Invalid input:", e)
        return

    ipos_by_name, scored = load_data()
    candidates = build_candidates(ipos_by_name, scored, hold_date)
    if not candidates:
        print("No eligible IPOs found.")
        return

    print(f"✅ Found {len(candidates)} eligible IPOs:\n")
    for c in candidates:
        print(f"• {c['ipo']} | {c['category']} | Score {c['composite']} | Min ₹{int(c['min_invest'])} | Retail {c['retail_quota']}%")

    allocation, remaining = allocate_balanced(candidates, budget)
    if not allocation:
        print("❌ No allocation possible with given budget.")
        return

    total_invested = sum(a["invested"] for a in allocation)

    print("\n📈 Optimized Allocation Plan:")
    for a in allocation:
        print(f"  - {a['ipo']}: ₹{int(a['invested'])} ({a['lots']} lot(s)) | Score {a['composite']} | Retail {a['retail_quota']}%")
    print(f"\n💵 Total Invested: ₹{int(total_invested)}")
    print(f"💤 Remaining: ₹{int(remaining)}")

    # save results safely
    rec = {
        "created_at": datetime.utcnow().isoformat(),
        "budget": budget,
        "hold_until": hold_date.isoformat(),
        "allocation": allocation,
        "total_invested": total_invested,
        "leftover": remaining,
        "count": len(allocation)
    }
    rec = sanitize_for_mongo(rec)

    client = MongoClient(MONGO_URI)
    client[DB_NAME][COL_RECOMMEND].insert_one(rec)
    client.close()
    print("\n✅ Saved recommendation to MongoDB successfully.")

if __name__ == "__main__":
    main()


📊 Balanced IPO Allocator (Retail)

✅ Found 6 eligible IPOs:

• PhysicsWallah | Mainboard | Score 6.3 | Min ₹14933 | Retail 10.0%
• Emmvee Photovoltaic Power | Mainboard | Score 6.171 | Min ₹14973 | Retail 10.0%
• Workmates Core2Cloud | SME | Score 6.135 | Min ₹244800 | Retail 10.0%
• Tenneco Clean Air | Mainboard | Score 6.089 | Min ₹14689 | Retail 10.0%
• Shining Tools | SME | Score 6.028 | Min ₹273600 | Retail 10.0%
• Shreeji Global FMCG | SME | Score 5.91 | Min ₹250000 | Retail 10.0%
⚠️ PuLP not installed, using fallback greedy mode.

📈 Optimized Allocation Plan:
  - PhysicsWallah: ₹44799 (3 lot(s)) | Score 6.3 | Retail 10.0%
  - Tenneco Clean Air: ₹44067 (3 lot(s)) | Score 6.089 | Retail 10.0%
  - Emmvee Photovoltaic Power: ₹44919 (3 lot(s)) | Score 6.171 | Retail 10.0%
  - Workmates Core2Cloud: ₹734400 (3 lot(s)) | Score 6.135 | Retail 10.0%

💵 Total Invested: ₹868185
💤 Remaining: ₹131815

✅ Saved recommendation to MongoDB successfully.


In [7]:
import os
import re
from datetime import datetime, date
from pymongo import MongoClient

# ---------- CONFIG ----------
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017")
DB_NAME = "ipo_db"
COL_IPOS = "ipos"
COL_ANALYSIS = "ipo_analysis"
COL_RECOMMEND = "ipo_portfolio_recommendations"

MIN_INVEST_MAINBOARD = 15000
DEFAULT_MAX_LOTS_PER_IPO = 3   # soft cap for diversification initially
DIVERSIFICATION_WEIGHT = 0.10  # penalty in MILP objective
TOP_FILL_K = 3                 # when filling leftovers, prioritize top K IPOs

# ---------- UTILITIES ----------
def safe_float(x):
    try:
        return float(str(x).replace("₹", "").replace(",", "").strip())
    except Exception:
        return None

def try_parse_date(s):
    if not s:
        return None
    s = str(s).strip()
    fmts = ["%Y-%m-%d", "%d-%b-%Y", "%d %b %Y", "%d/%m/%Y"]
    for f in fmts:
        try:
            return datetime.strptime(s[:len(f)], f).date()
        except Exception:
            pass
    try:
        import dateutil.parser
        return dateutil.parser.parse(s, dayfirst=True).date()
    except Exception:
        return None

def parse_lot_and_min_invest(text):
    if not text:
        return None, None
    s = str(text)
    m = re.search(r"(?:Min[:\s]*)?(\d{1,6})\s*shares?.*?₹\s?([\d,]+)", s, flags=re.I)
    if m:
        lot = safe_float(m.group(1))
        min_inv = safe_float(m.group(2))
        return int(lot) if lot else None, float(min_inv) if min_inv else None
    m2 = re.search(r"₹\s?([\d,]+)", s)
    if m2:
        return None, float(safe_float(m2.group(1)))
    return None, None

def parse_issue_mid(ipo_doc):
    v = ipo_doc.get("issue_price") or ipo_doc.get("extracted_fields", {}).get("Price Band")
    if isinstance(v, dict):
        mid = v.get("avg") or v.get("mid") or v.get("min")
        return safe_float(mid)
    if not v:
        return None
    nums = re.findall(r"\d+\.?\d*", str(v))
    nums = [safe_float(n) for n in nums if safe_float(n) is not None]
    return sum(nums)/len(nums) if nums else None

def sanitize_for_mongo(obj):
    """Recursively convert datetime.date to ISO string and datetime to iso."""
    if isinstance(obj, dict):
        return {k: sanitize_for_mongo(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [sanitize_for_mongo(x) for x in obj]
    elif isinstance(obj, date):
        return datetime(obj.year, obj.month, obj.day).isoformat()
    elif isinstance(obj, datetime):
        return obj.isoformat()
    else:
        return obj

# ---------- SCORING COMPONENTS ----------
def extract_retail_quota(text):
    if not text:
        return 10.0
    m = re.search(r"Retail\s*:?(\d+\.?\d*)%", str(text), flags=re.I)
    if m:
        return safe_float(m.group(1))
    return 10.0

def extract_fundamental_score(text):
    if not text:
        return 5.0
    score = 5.0
    t = str(text).lower()
    if "profit" in t or "positive" in t or "growth" in t:
        score += 2
    if "loss" in t or "negative" in t:
        score -= 2
    # ROE, D/E, EPS heuristics:
    m = re.search(r"roe[:\s]*([-\d.]+)", t)
    if m:
        roe = safe_float(m.group(1))
        if roe and roe > 10:
            score += 1.5
    m = re.search(r"d/?e[:\s]*([-\d.]+)", t)
    if m:
        de = safe_float(m.group(1))
        if de and de > 1:
            score -= 1
    m = re.search(r"eps[:\s]*[-(₹]?([\d.]+)", t)
    if m:
        eps = safe_float(m.group(1))
        if eps and eps > 0:
            score += 1
        else:
            score -= 1
    return max(1, min(score, 10))

def extract_sentiment(text):
    if not text:
        return 5.0
    t = str(text).lower()
    score = 5.0
    good = ["growing", "leader", "expanding", "innovative", "strong", "profitable", "stable"]
    bad = ["loss", "decline", "volatile", "uncertain", "risky", "unprofitable"]
    for w in good:
        if w in t:
            score += 0.5
    for w in bad:
        if w in t:
            score -= 0.5
    return max(1, min(score, 10))

def compute_composite_and_breakdown(ipo_doc, analysis):
    """
    Returns:
      composite (float),
      breakdown: dict with base_score, retail_quota, rq_score, fund_score, sentiment, gmp_strength, formula_weights
    """
    fields = ipo_doc.get("extracted_fields", {}) or {}
    base = analysis.get("score", 5)  # from earlier scorer
    retail = extract_retail_quota(fields.get("Investor Quota Split"))
    fund = extract_fundamental_score(fields.get("Valuation Ratios (EPS, ROE, ROCE, D/E, NAV)") or fields.get("Financial Performance (FY23–FY25)"))
    sent = extract_sentiment(fields.get("Company Overview"))
    gmp = safe_float(ipo_doc.get("gmp_investorgain")) or 0
    issue = parse_issue_mid(ipo_doc)
    gmp_strength = (gmp / issue * 100) if (gmp and issue) else 0
    # retail quota normalized to 0-10
    rq_score = min(retail / 10, 1) * 10

    # weights (documented to user later)
    w_base = 0.30
    w_rq = 0.25
    w_fund = 0.20
    w_gmp = 0.15
    w_sent = 0.10

    composite = w_base*base + w_rq*rq_score + w_fund*fund + w_gmp*(gmp_strength/10) + w_sent*sent
    composite = round(min(composite, 10), 3)

    breakdown = {
        "base_score": base,
        "retail_quota_pct": retail,
        "rq_score": round(rq_score,3),
        "fund_score": round(fund,3),
        "sentiment_score": round(sent,3),
        "gmp_strength_pct": round(gmp_strength,3),
        "weights": {"base": w_base, "retail": w_rq, "fund": w_fund, "gmp": w_gmp, "sentiment": w_sent}
    }
    return composite, breakdown

# ---------- DATA LOAD ----------
def load_data():
    client = MongoClient(MONGO_URI)
    ipos = list(client[DB_NAME][COL_IPOS].find({}))
    analysis = list(client[DB_NAME][COL_ANALYSIS].find({}))
    client.close()
    ipos_by_name = {d["ipo"]: d for d in ipos if "ipo" in d}
    scored = {a["ipo"]: a for a in analysis if "ipo" in a}
    return ipos_by_name, scored

# ---------- BUILD CANDIDATES ----------
def build_candidates(ipos_by_name, scored, hold_date):
    cands = []
    for name, ipo in ipos_by_name.items():
        if name not in scored:
            continue
        analysis = scored[name]
        if analysis.get("status") != "scored" and "score" not in analysis:
            continue
        fields = ipo.get("extracted_fields", {}) or {}
        close = ipo.get("close_date") or fields.get("IPO Dates") or fields.get("Close Date")
        close_date = try_parse_date(str(close))
        if not close_date or close_date > hold_date:
            continue

        issue_mid = parse_issue_mid(ipo)
        if not issue_mid:
            continue

        lot, min_inv = parse_lot_and_min_invest(fields.get("Market Lot & Amounts"))
        if not min_inv:
            min_inv = (lot * issue_mid) if lot else MIN_INVEST_MAINBOARD

        composite, breakdown = compute_composite_and_breakdown(ipo, analysis)
        if composite < 5:
            # skip per user instruction
            continue

        cands.append({
            "ipo": name,
            "category": ipo.get("category", "Mainboard"),
            "composite": composite,
            "breakdown": breakdown,
            "issue_mid": issue_mid,
            "lot": int(lot) if lot else None,
            "min_invest": float(min_inv),
            "close_date": close_date,
            "gmp_investorgain": ipo.get("gmp_investorgain"),
            "analysis": analysis
        })
    return cands

# ---------- GREEDY-FILL (improved) ----------
def greedy_fill_full(candidates, budget):
    """
    Improved greedy:
     - initial pass: allocate 1 lot to as many top candidates as possible (descending composite/min_invest)
     - second pass: try to add additional lots to top K candidates in round-robin until can't
     - final pass: try any candidate that can fit another lot
    This aggressively uses budget to minimize leftover while respecting lot units.
    """
    candidates = sorted(candidates, key=lambda x: x["composite"]/x["min_invest"], reverse=True)
    allocation = []
    remaining = budget
    min_unit = min(c["min_invest"] for c in candidates)

    # initial one-lot diversification
    for c in candidates:
        if remaining >= c["min_invest"]:
            allocation.append({"ipo": c["ipo"], "lots": 1, "min_invest": c["min_invest"], "invested": c["min_invest"], "composite": c["composite"]})
            remaining -= c["min_invest"]

    # second pass: fill top-K repeatedly (this prevents huge leftover)
    top_k = candidates[:min(TOP_FILL_K, len(candidates))]
    # keep adding 1 lot to each top_k in order while possible
    added = True
    while added and remaining >= min_unit:
        added = False
        for c in top_k:
            if remaining >= c["min_invest"]:
                found = next((a for a in allocation if a["ipo"] == c["ipo"]), None)
                if found:
                    found["lots"] += 1
                    found["invested"] += c["min_invest"]
                else:
                    allocation.append({"ipo": c["ipo"], "lots": 1, "min_invest": c["min_invest"], "invested": c["min_invest"], "composite": c["composite"]})
                remaining -= c["min_invest"]
                added = True
            if remaining < min_unit:
                break

    # final pass: try to spend remaining on any candidate by composite/unit
    while remaining >= min_unit:
        affordable = [c for c in candidates if c["min_invest"] <= remaining]
        if not affordable:
            break
        pick = max(affordable, key=lambda x: x["composite"]/x["min_invest"])
        found = next((a for a in allocation if a["ipo"] == pick["ipo"]), None)
        if found:
            found["lots"] += 1
            found["invested"] += pick["min_invest"]
        else:
            allocation.append({"ipo": pick["ipo"], "lots": 1, "min_invest": pick["min_invest"], "invested": pick["min_invest"], "composite": pick["composite"]})
        remaining -= pick["min_invest"]

    # sort allocation by composite desc
    allocation = sorted(allocation, key=lambda x: x["composite"], reverse=True)
    return allocation, remaining

# ---------- MILP (balanced) ----------
def allocate_balanced(candidates, budget):
    """Try PuLP MILP with diversification penalty; fallback to greedy_fill_full if PuLP absent."""
    try:
        import pulp
    except ImportError:
        return None, None  # caller will pick greedy

    prob = pulp.LpProblem("Balanced_IPO", pulp.LpMaximize)
    vars_map = {}
    for c in candidates:
        safe_name = re.sub(r"\W+", "_", c["ipo"])
        max_possible = int(max(1, budget // c["min_invest"]))
        cap = min(DEFAULT_MAX_LOTS_PER_IPO, max_possible)
        vars_map[c["ipo"]] = pulp.LpVariable(f"lots_{safe_name}", lowBound=0, upBound=cap, cat="Integer")

    # objective: maximize sum(score * lots) - diversification_penalty
    obj = pulp.lpSum([c["composite"] * vars_map[c["ipo"]] for c in candidates])
    # diversification penalty uses squared lots to penalize concentration
    penalty = DIVERSIFICATION_WEIGHT * pulp.lpSum([(vars_map[c["ipo"]]**2) for c in candidates])
    prob += obj - penalty

    # budget constraint
    prob += pulp.lpSum([c["min_invest"] * vars_map[c["ipo"]] for c in candidates]) <= budget

    # solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=10))

    allocation = []
    total_invested = 0.0
    for c in candidates:
        v = int(pulp.value(vars_map[c["ipo"]]) or 0)
        if v > 0:
            invested = v * c["min_invest"]
            allocation.append({"ipo": c["ipo"], "lots": v, "invested": invested, "min_invest": c["min_invest"], "composite": c["composite"]})
            total_invested += invested
    remaining = budget - total_invested
    return allocation, remaining

# ---------- EXPLAINABILITY ----------
def explain_allocation(allocation, candidates_dict):
    explain = {}
    for a in allocation:
        c = candidates_dict.get(a["ipo"])
        reasons_more = []
        reasons_less = []
        br = c["breakdown"]
        # add numeric breakdown
        reasons_more.append(f"Composite score {c['composite']} computed from base_score={br['base_score']}, retail_q={br['retail_quota_pct']}%, fund={br['fund_score']}, sentiment={br['sentiment_score']}, gmp_str={br['gmp_strength_pct']}% using weights {br['weights']}.")
        if br['gmp_strength_pct'] > 10:
            reasons_more.append(f"High GMP strength {br['gmp_strength_pct']}% → strong listing expectation.")
        if br['retail_quota_pct'] >= 30:
            reasons_more.append("High retail quota → better allotment odds.")
        else:
            reasons_less.append(f"Retail quota {br['retail_quota_pct']}% is low; allotment probability may be limited.")
        if br['fund_score'] >= 6:
            reasons_more.append("Fundamentals show positive indicators.")
        else:
            reasons_less.append("Fundamentals are weak/moderate.")
        if "sme" in str(c.get("category","")).lower():
            reasons_less.append("SME IPO: higher risk & lower liquidity — allocate small exposure.")
        explain[a["ipo"]] = {"reasons_more": reasons_more, "reasons_less": reasons_less, "breakdown": br}
    return explain

# ---------- MAIN ----------
def main():
    print("📊 IPO Allocator — Full explainable mode\n")
    try:
        budget = float(input("Enter total budget (₹): ").strip())
        hold_date = datetime.strptime(input("Enter hold-until date (YYYY-MM-DD): ").strip(), "%Y-%m-%d").date()
    except Exception as e:
        print("Invalid input:", e)
        return

    ipos_by_name, scored = load_data()
    candidates = build_candidates(ipos_by_name, scored, hold_date)
    if not candidates:
        print("No eligible IPOs (score >=5 & within hold date). Exiting.")
        return

    print(f"✅ Found {len(candidates)} eligible IPOs:\n")
    for c in candidates:
        print(f"• {c['ipo']} | {c['category']} | Composite {c['composite']} | Min ₹{int(c['min_invest'])} | Retail {c['breakdown']['retail_quota_pct']}%")

    # Try MILP; if not possible or leftover too high, fallback to improved greedy that aggressively consumes budget
    allocation, remaining = allocate_balanced(candidates, budget)
    if allocation is None:
        print("⚠️ PuLP not installed or failed — using improved greedy filler.")
        allocation, remaining = greedy_fill_full(candidates, budget)
    else:
        # if remaining is too large ( > 1% budget ), try aggressive fill to reduce leftover
        if remaining > 0.01 * budget:
            g_alloc, g_rem = greedy_fill_full(candidates, budget)
            # pick one with smaller leftover; if tie, pick one with larger total composite*lots
            def used(a): return sum(x["invested"] for x in a)
            if (budget - g_rem) > (budget - remaining):
                allocation, remaining = g_alloc, g_rem

    total_invested = sum(a["invested"] for a in allocation)
    print("\n📈 Final Allocation Plan:")
    for a in allocation:
        print(f"  - {a['ipo']}: ₹{int(a['invested'])} ({a['lots']} lots) | min_unit=₹{int(a['min_invest'])} | score {a['composite']}")
    print(f"\n💵 Total Invested: ₹{int(total_invested)}")
    print(f"💤 Remaining: ₹{int(remaining)}")

    # explain
    candidates_dict = {c["ipo"]: c for c in candidates}
    explain = explain_allocation(allocation, candidates_dict)

    print("\n--- Explainability & Formulas ---")
    print("\nAlgorithm summary:")
    print("  - We compute a composite score per IPO combining:")
    print("      composite = 0.30*base_score + 0.25*rq_score + 0.20*fund_score + 0.15*(gmp_strength/10) + 0.10*sentiment_score")
    print("    where rq_score = min(retail_pct/10,1)*10 (normalized 0-10).")
    print("  - Primary solver: MILP (PuLP) maximizing sum(composite * lots) minus DIVERSIFICATION_WEIGHT * sum(lots^2).")
    print("  - Fallback: improved greedy that:")
    print("      1) gives 1 lot to each top candidate by composite/unit if affordable,")
    print("      2) repeatedly fills top K candidates (round-robin) to use leftover,")
    print("      3) finally uses any remaining budget on best composite/unit candidate.")
    print("  - Unit = min_invest (usually lot * issue price or ₹15k for mainboard fallback).")
    print("  - We skip any IPO with composite < 5 per your instruction.\n")

    print("Parameters used:")
    print(f"  - MIN_INVEST_MAINBOARD = {MIN_INVEST_MAINBOARD}")
    print(f"  - DEFAULT_MAX_LOTS_PER_IPO = {DEFAULT_MAX_LOTS_PER_IPO}")
    print(f"  - DIVERSIFICATION_WEIGHT = {DIVERSIFICATION_WEIGHT}")
    print(f"  - TOP_FILL_K = {TOP_FILL_K}\n")

    print("Per-IPO reasons (why more / why less):")
    for ipo, info in explain.items():
        print(f"\n🔎 {ipo}")
        br = info["breakdown"]
        print(f"  Breakdown: base={br['base_score']}, retail%={br['retail_quota_pct']}%, rq_score={br['rq_score']}, fund={br['fund_score']}, sentiment={br['sentiment_score']}, gmp_str%={br['gmp_strength_pct']}")
        if info["reasons_more"]:
            print("  Reasons to invest more:")
            for r in info["reasons_more"]:
                print("   -", r)
        if info["reasons_less"]:
            print("  Reasons to be cautious:")
            for r in info["reasons_less"]:
                print("   -", r)

    # persist recommendation (sanitize dates)
    rec = {
        "created_at": datetime.utcnow().isoformat(),
        "budget": budget,
        "hold_until": hold_date.isoformat(),
        "allocation": allocation,
        "explain": explain,
        "total_invested": total_invested,
        "leftover": remaining
    }
    rec = sanitize_for_mongo(rec)
    client = MongoClient(MONGO_URI)
    client[DB_NAME][COL_RECOMMEND].insert_one(rec)
    client.close()
    print("\n✅ Saved recommendation to MongoDB (collection:", COL_RECOMMEND, ")")

if __name__ == "__main__":
    main()


📊 IPO Allocator — Full explainable mode

✅ Found 6 eligible IPOs:

• PhysicsWallah | Mainboard | Composite 6.3 | Min ₹14933 | Retail 10.0%
• Emmvee Photovoltaic Power | Mainboard | Composite 6.171 | Min ₹14973 | Retail 10.0%
• Workmates Core2Cloud | SME | Composite 6.135 | Min ₹244800 | Retail 10.0%
• Tenneco Clean Air | Mainboard | Composite 6.089 | Min ₹14689 | Retail 10.0%
• Shining Tools | SME | Composite 6.028 | Min ₹273600 | Retail 10.0%
• Shreeji Global FMCG | SME | Composite 5.91 | Min ₹250000 | Retail 10.0%
⚠️ PuLP not installed or failed — using improved greedy filler.

📈 Final Allocation Plan:
  - PhysicsWallah: ₹74665 (5 lots) | min_unit=₹14933 | score 6.3
  - Emmvee Photovoltaic Power: ₹74865 (5 lots) | min_unit=₹14973 | score 6.171
  - Workmates Core2Cloud: ₹244800 (1 lots) | min_unit=₹244800 | score 6.135
  - Tenneco Clean Air: ₹73445 (5 lots) | min_unit=₹14689 | score 6.089
  - Shining Tools: ₹273600 (1 lots) | min_unit=₹273600 | score 6.028
  - Shreeji Global FMCG: ₹25